# Construir la base de datos / dataframe completo
La idea de este jupyter es construir ya sea la base de datos de manera que todas las columnas casen con todos los datasets (en caso que un dataset tenga columnas faltantes entonces rellenar esa columna con nulos) y construir la base de datos o el dataframe, después lo decidiré

## Importar Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import os
import re

pd.set_option('future.no_silent_downcasting', True)

## Cargar los datasets

In [3]:
def load_all_datasets():
    datasets = {}
    directory = f"./dataset/Datos/"
    files = os.listdir(directory)
    files.sort()
    for file in files:
        if file.endswith(".sav"):
            data = pd.read_spss(directory + file)  
            datasets[file.split(".")[0][-4:]] = data[sorted(data.columns)]
    return datasets

In [4]:
datasets = load_all_datasets()
datasets.keys()

dict_keys(['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])

## Revisar que columnas faltan en cada dataset

In [5]:
def get_columns_completed(datasets):
    columns = set()
    for name, dataset  in sorted(datasets.items(), reverse=True):
        columns.update(dataset.columns)
    return columns

def check_lacking_columns(datasets, columns):
    for dataset in datasets:
        for column in columns:
            if column not in datasets[dataset].columns:
                print(f"{dataset} does not have column {column}")

In [6]:
complete_columns = get_columns_completed(datasets=datasets)
print(f"Total de columnas: {len(complete_columns)} vs Total de columnas según el dataset más reciente: {len(datasets['2023'].columns)}")

Total de columnas: 77 vs Total de columnas según el dataset más reciente: 75


Esto lo que indica es que habrán algunas columnas que son repetidas, es decir que la estrategia de utilizar complete_columns para poder unificar los datasets no es efectiva y habrá que cambiar la estrategia.

In [7]:
def check_which_colums_are_missing_in_which_dataset(datasets, columns):
    lacking_columns = {}
    for name, dataset in sorted(datasets.items(), reverse=True):
        for column in columns:
            if column not in dataset.columns:
                if name not in lacking_columns:
                    lacking_columns[name] = []
                lacking_columns[name].append(column)
    return lacking_columns
check_which_colums_are_missing_in_which_dataset(datasets, complete_columns)

{'2023': ['filter_$', 'AGR_GRUPET'],
 '2022': ['DEPTO', 'HEC_DEPTO', 'filter_$', 'AGR_GRUPET'],
 '2021': ['DEPTO', 'HEC_DEPTO', 'filter_$', 'AGR_GRUPET'],
 '2020': ['NUMERO_BOLETA', 'DEPTO', 'HEC_DEPTO', 'filter_$', 'AGR_GRUPET'],
 '2019': ['NUMERO_BOLETA', 'DEPTO', 'HEC_DEPTO', 'filter_$', 'AGR_GRUPET'],
 '2018': ['NUMERO_BOLETA', 'DEPTO', 'HEC_DEPTO', 'filter_$', 'AGR_GRUPET'],
 '2017': ['NUMERO_BOLETA', 'DEPTO', 'HEC_DEPTO', 'AGR_GRUPET'],
 '2016': ['ARTICULOCODPEN1',
  'ARTICULOVCM4',
  'NUMERO_BOLETA',
  'ARTICULOTRAS1',
  'DEPTO',
  'ARTICULOCODPEN2',
  'HEC_DEPTO',
  'CONDUCENTE',
  'ARTICULOVCM1',
  'ARTICULOVIF3',
  'ARTICULOTRAS3',
  'ARTICULOTRAS4',
  'filter_$',
  'ARTICULOVIF2',
  'ORGANISMO_REMITE',
  'ARTICULOVIF4',
  'ARTICULOCODPEN4',
  'ARTICULOVIF1',
  'AGR_GRUPET',
  'ORGANISMO_JURISDICCIONAL',
  'ARTICULOTRAS2',
  'ARTICULOVCM2',
  'ARTICULOVCM3',
  'ARTICULOCODPEN3'],
 '2015': ['ARTICULOCODPEN1',
  'ARTICULOVCM4',
  'NUMERO_BOLETA',
  'ARTICULOTRAS1',
  'DEPTO',
 

Investigando he visto que hay algunas columnas que tienen su nombre mal escrito, por ejemplo, para el dataset de 2023 AGR_GURPET en realidad debería de escribirse como AGR_GRUPET.

Así mismo, hay otras columnas que no existen pero que son obtenibles a partir del dataset que si contiene las columnas como DEPTO y HEC_DEPTO, que el primero es el departamento del registro y el segundo es el departamento del hecho.

### Eliminar las columnas que son innecesarias
Se deben de eliminar columnas como número de boleta que realmente no es necesaria dado que es el número de solicitud pero realmente no influye. Así mismo, el filter que tampoco hace algo.

In [8]:
def drop_column_in_datasets(column):
    for name, dataset in datasets.items():
        if column in dataset.columns:
            dataset.drop(column, axis=1, inplace=True)

In [9]:
drop_column_in_datasets("NUMERO_BOLETA")
drop_column_in_datasets('filter_$')

### Renombrar columnas
Se tienen que renombrar algunas columnas como AGR_GURPET que realmente es lo mismo que AGR_GRUPET

In [10]:
def rename_columns(dataset, renaming_columns):
    new_names = {key: key for key in dataset.columns}
    new_names.update(renaming_columns)
    return new_names
def rename_columns_in_datasets(renaming_columns):
    for name, dataset in datasets.items():
        new_names = rename_columns(dataset, renaming_columns)
        dataset.rename(columns=new_names, inplace=True)

In [11]:
rename_columns_in_datasets({"AGR_GURPET": "AGR_GRUPET"})

### Obtener el municipio y el departamento para los hechos y el registro
Esta es una función para obtener un dataframe con el municipio y el departamento de manera que para los demás datasets se puedan obtener para los hechos y el registro ya que no están registrados los departamentos

In [12]:
def get_df_for_munic_dept():
    dfs = []
    for index, dataset in datasets.items():
        if "DEPTO" in dataset.columns:
            df0 = pd.DataFrame()
            df0["DEPTO"] = dataset["DEPTO"]
            df0["MUNIC"] = dataset["DEPTO_MCPIO"]
            df1 = pd.DataFrame()
        if "HEC_DEPTO" in dataset.columns:
            df1["DEPTO"] = dataset["HEC_DEPTO"]
            df1["MUNIC"] = dataset["HEC_DEPTOMCPIO"]
            dfs.append(pd.concat([df0.dropna(), df1.dropna()], axis=0).drop_duplicates())
    return pd.concat(dfs, axis=0).drop_duplicates()
dfDeptoMunic = get_df_for_munic_dept()
dfDeptoMunic

,DEPTO,MUNIC
0,Alta Verapaz,Cobán
1,Guatemala,Mixco
2,Quetzaltenango,Sibilia
3,Chimaltenango,San Andrés Itzapa
4,Jutiapa,Jutiapa
...,...,...
4971,San Marcos,El Quetzal
11527,Santa Rosa,Taxisco
12713,Huehuetenango,La Libertad
14061,Sacatepéquez,Santo Domingo Xenacoj


In [13]:
def generate_department_column(datasets, dfDeptoMunic):
    # Create a mapping from municipios to departments
    munic_to_depto = dfDeptoMunic.set_index("MUNIC")["DEPTO"].to_dict()

    for year, df in datasets.items():
        if year == "2023":
            continue  # Skip 2023 dataset
        if "DEPTO_MCPIO" in df.columns:
            df["DEPTO"] = df["DEPTO_MCPIO"].map(munic_to_depto)
        if "HEC_DEPTOMCPIO" in df.columns:
            df["HEC_DEPTO"] = df["HEC_DEPTOMCPIO"].map(munic_to_depto)
generate_department_column(datasets, dfDeptoMunic)

### Llenar con nulos las columnas que hacen falta


In [14]:
complete_columns = get_columns_completed(datasets=datasets)
columns_missing = check_which_colums_are_missing_in_which_dataset(datasets, complete_columns)
columns_missing

{'2016': ['ARTICULOCODPEN1',
  'ARTICULOCODPEN4',
  'CONDUCENTE',
  'ARTICULOVCM1',
  'ARTICULOVIF1',
  'ARTICULOVCM4',
  'ARTICULOTRAS2',
  'ARTICULOVIF3',
  'ARTICULOVCM2',
  'ARTICULOTRAS3',
  'ARTICULOTRAS4',
  'ARTICULOTRAS1',
  'ARTICULOVIF2',
  'ARTICULOVIF4',
  'ORGANISMO_REMITE',
  'ARTICULOCODPEN2',
  'ORGANISMO_JURISDICCIONAL',
  'ARTICULOVCM3',
  'ARTICULOCODPEN3'],
 '2015': ['ARTICULOCODPEN1',
  'ARTICULOCODPEN4',
  'CONDUCENTE',
  'ARTICULOVCM1',
  'ARTICULOVIF1',
  'ARTICULOVCM4',
  'ARTICULOTRAS2',
  'ARTICULOVIF3',
  'ARTICULOVCM2',
  'ARTICULOTRAS3',
  'ARTICULOTRAS4',
  'ARTICULOTRAS1',
  'ARTICULOVIF2',
  'ARTICULOVIF4',
  'ORGANISMO_REMITE',
  'ARTICULOCODPEN2',
  'ORGANISMO_JURISDICCIONAL',
  'ARTICULOVCM3',
  'ARTICULOCODPEN3'],
 '2014': ['ARTICULOCODPEN1',
  'ARTICULOCODPEN4',
  'CONDUCENTE',
  'ARTICULOVCM1',
  'ARTICULOVIF1',
  'ARTICULOVCM4',
  'ARTICULOTRAS2',
  'ARTICULOVIF3',
  'ARTICULOVCM2',
  'ARTICULOTRAS3',
  'ARTICULOTRAS4',
  'ARTICULOTRAS1',
  'ART

In [15]:
def generate_nulls_for_missing_columns(datasets, columns_missing):
    for name, dataset in datasets.items():
        try:
            
            missing_columns = columns_missing[name]
            for column in missing_columns:
                dataset[column] = np.nan
        except:
            pass
generate_nulls_for_missing_columns(datasets, columns_missing)

In [16]:
complete_columns = get_columns_completed(datasets=datasets)
columns_missing = check_which_colums_are_missing_in_which_dataset(datasets, complete_columns)
columns_missing

{}

## Corregir los datos categóricos

In [17]:
def replace_values_in_column(column, values):
    for dataset in datasets.values():
        if column in dataset.columns:
            dataset[column] = dataset[column].replace(values)

In [18]:
replace_values_in_column("VIC_ALFAB", {'Si': "Alfabeta", "No": "Analfabeta"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [19]:
replace_values_in_column("AGR_ALFAB", {'Si': "Alfabeta", "No": "Analfabeta"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [20]:
replace_values_in_column(
    "AGR_ESCOLARIDAD", 
    {
        10.0: "Ninguno", 
        21.0: "Primero Primaria", 
        22.0: "Segundo Primaria", 
        23.0: "Tercero Primaria", 
        24.0: "Cuarto Primaria", 
        25.0: "Quinto Primaria", 
        26.0: "Sexto Primaria",
        29.0: "Primaria Grado Ignorado",
        31.0: "Primero Basico",
        32.0: "Segundo Basico",
        33.0: "Tercero Basico",
        39.0: "Basico Grado Ignorado",
        44.0: "Cuarto Diversificado",
        45.0: "Quinto Diversificado",
        46.0: "Sexto Diversificado",
        49.0: "Diversificado Grado Ignorado",
        51.0: "Primer Ano Universitario",
        52.0: "Segundo Ano Universitario",
        53.0: "Tercer Ano Universitario",
        54.0: "Cuarto Ano Universitario",
        55.0: "Quinto Ano Universitario",
        56.0: "Sexto Ano Universitario",
        57.0: "Septimo Ano Universitario",
        59.0: "Universitario Grado Ignorado",
        99.0: "Ignorado",
        "Cuarto año universitario": "Cuarto Ano Universitario",
        "Básico grado ignorado": "Basico Grado Ignorado",
        "Cuarto año universitario":"Cuarto Ano Universitario",
        "Cuarto diversificado":"Cuarto Diversificado",
        "Cuarto primaria":"Cuarto Primaria",
        "Diversificado grado ignorado":"Diversificado Grado Ignorado",
        "Grado universitario ignorado":"Universitario Grado Ignorado",
        "Ignorado año y nivel de escolaridad": "Ignorado",
        "Ninguna": "Ninguno",
        "Primaria grado ignorado": "Primaria Grado Ignorado",
        "Primer año universitario": "Primer Ano Universitario",
        "Primero básico": "Primero Basico",
        "Primero primaria": "Primero Primaria",
        "Quinto año universitario": "Quinto Ano Universitario",
        "Quinto diversificado": "Quinto Diversificado",
        "Quinto primaria": "Quinto Primaria",
        "Segundo año universitario": "Segundo Ano Universitario",
        "Segundo básico": "Segundo Basico",
        "Segundo primaria": "Segundo Primaria",
        "Septimo año universitario": "Septimo Ano Universitario",
        "Sexto año universitario": "Sexto Ano Universitario",
        "Sexto diversificado": "Sexto Diversificado",
        "Sexto primaria": "Sexto Primaria",
        "Tercer año universitario": "Tercer Ano Universitario",
        "Tercero básico": "Tercero Basico",
        "Tercero primaria": "Tercero Primaria",
    }
    )

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [21]:
replace_values_in_column(
    "VIC_ESCOLARIDAD", 
    {
        10.0: "Ninguno", 
        21.0: "Primero Primaria", 
        22.0: "Segundo Primaria", 
        23.0: "Tercero Primaria", 
        24.0: "Cuarto Primaria", 
        25.0: "Quinto Primaria", 
        26.0: "Sexto Primaria",
        29.0: "Primaria Grado Ignorado",
        31.0: "Primero Basico",
        32.0: "Segundo Basico",
        33.0: "Tercero Basico",
        39.0: "Basico Grado Ignorado",
        44.0: "Cuarto Diversificado",
        45.0: "Quinto Diversificado",
        46.0: "Sexto Diversificado",
        49.0: "Diversificado Grado Ignorado",
        51.0: "Primer Ano Universitario",
        52.0: "Segundo Ano Universitario",
        53.0: "Tercer Ano Universitario",
        54.0: "Cuarto Ano Universitario",
        55.0: "Quinto Ano Universitario",
        56.0: "Sexto Ano Universitario",
        57.0: "Septimo Ano Universitario",
        59.0: "Universitario Grado Ignorado",
        99.0: "Ignorado",
        "Cuarto año universitario": "Cuarto Ano Universitario",
        "Básico grado ignorado": "Basico Grado Ignorado",
        "Cuarto año universitario":"Cuarto Ano Universitario",
        "Cuarto diversificado":"Cuarto Diversificado",
        "Cuarto primaria":"Cuarto Primaria",
        "Diversificado grado ignorado":"Diversificado Grado Ignorado",
        "Grado universitario ignorado":"Universitario Grado Ignorado",
        "Ignorado año y nivel de escolaridad": "Ignorado",
        "Ninguna": "Ninguno",
        "Primaria grado ignorado": "Primaria Grado Ignorado",
        "Primer año universitario": "Primer Ano Universitario",
        "Primero básico": "Primero Basico",
        "Primero primaria": "Primero Primaria",
        "Quinto año universitario": "Quinto Ano Universitario",
        "Quinto diversificado": "Quinto Diversificado",
        "Quinto primaria": "Quinto Primaria",
        "Segundo año universitario": "Segundo Ano Universitario",
        "Segundo básico": "Segundo Basico",
        "Segundo primaria": "Segundo Primaria",
        "Septimo año universitario": "Septimo Ano Universitario",
        "Sexto año universitario": "Sexto Ano Universitario",
        "Sexto diversificado": "Sexto Diversificado",
        "Sexto primaria": "Sexto Primaria",
        "Tercer año universitario": "Tercer Ano Universitario",
        "Tercero básico": "Tercero Basico",
        "Tercero primaria": "Tercero Primaria",
    }
    )

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [22]:
replace_values_in_column("VIC_GRUPET", {'Xincas': "Xinka", "Xinca": "Xinka", "Xinkas": "Xinka", "Otros": "Otro", "Mayas": "Maya", "Garifunas":"Garifuna"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [23]:
replace_values_in_column("AGR_GRUPET", {'Xincas': "Xinka", "Xinca": "Xinka", "Xinkas": "Xinka", "Otros": "Otro", "Mayas": "Maya", "Garifunas":"Garifuna"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [24]:

replace_values_in_column("AGR_TRABAJA", {'No trabaja por un salario o ingreso': "No", "Si trabaja por un salario o ingreso": "Si"})
replace_values_in_column("VIC_TRABAJA", {'No trabaja por un salario o ingreso': "No", "Si trabaja por un salario o ingreso": "Si"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [25]:
replace_values_in_column("DEPTO_MCPIO", {
101.0:"Guatemala",
102.0:"Santa Catarina Pinula",
103.0:"San José Pinula",
104.0:"San José del Golfo",
105.0:"Palencia",
106.0:"Chinautla",
107.0:"San Pedro Ayampuc",
108.0:"Mixco",
109.0:"San Pedro Sacatepéquez",
110.0:"San Juan Sacatepéquez",
111.0:"San Raymundo",
112.0:"Chuarrancho",
113.0:"Fraijanes",
114.0:"Amatitlán",
115.0:"Villa Nueva",
116.0:"Villa Canales",
117.0:"Petapa",
201.0:"Guastatoya",
202.0:"Morazán",
203.0:"San Agustín Acasaguastlán",
204.0:"San Cristóbal Acasaguastlán",
205.0:"El Jícaro",
206.0:"Sansare",
207.0:"Sanarate",
208.0:"San Antonio La Paz",
301.0:"Antigua Guatemala",
302.0:"Jocotenango",
303.0:"Pastores",
304.0:"Sumpango",
305.0:"Santo Domingo Xenacoj",
306.0:"Santiago Sacatepéquez",
307.0:"San Bartolomé Milpas Altas",
308.0:"San Lucas Sacatepéquez",
309.0:"Santa Lucía Milpas Altas",
310.0:"Magdalena Milpas Altas",
311.0:"Santa María de Jesús",
312.0:"Ciudad Vieja",
313.0:"San Miguel Dueñas",
314.0:"Alotenango",
315.0:"San Antonio Aguas Calientes",
316.0:"Santa Catarina Barahona",
401.0:"Chimaltenango",
402.0:"San José Poaquil",
403.0:"San Martín Jilotepeque",
404.0:"Comalapa",
405.0:"Santa Apolonia",
406.0:"Tecpán Guatemala",
407.0:"Patzún",
408.0:"Pochuta",
409.0:"Patzicía",
410.0:"Santa Cruz Balanyá",
411.0:"Acatenango",
412.0:"Yepocapa",
413.0:"San Andrés Itzapa",
414.0:"Parramos",
415.0:"Zaragoza",
416.0:"El Tejar",
501.0:"Escuintla",
502.0:"Santa Lucía Cotzumalguapa",
503.0:"La Democracia",
504.0:"Siquinalá",
505.0:"Masagua",
506.0:"Tiquisate",
507.0:"La Gomera",
508.0:"Guanagazapa",
509.0:"San José",
510.0:"Iztapa",
511.0:"Palín",
512.0:"San Vicente Pacaya",
513.0:"Nueva Concepción",
514.0:"Sipacate",
601.0:"Cuilapa",
602.0:"Barberena",
603.0:"Santa Rosa de Lima",
604.0:"Casillas",
605.0:"San Rafael Las Flores",
606.0:"Oratorio",
607.0:"San Juan Tecuaco",
608.0:"Chiquimulilla",
609.0:"Taxisco",
610.0:"Santa María Ixhuatán",
611.0:"Guazacapán",
612.0:"Santa Cruz Naranjo",
613.0:"Pueblo Nuevo Viñas",
614.0:"Nueva Santa Rosa",
701.0:"Sololá",
702.0:"San José Chacayá",
703.0:"Santa María Visitación",
704.0:"Santa Lucía Utatlán",
705.0:"Nahualá",
706.0:"Santa Catarina Ixtahuacán",
707.0:"Santa Clara La Laguna",
708.0:"Concepción",
709.0:"San Andrés Semetabaj",
710.0:"Panajachel",
711.0:"Santa Catarina Palopó",
712.0:"San Antonio Palopó",
713.0:"San Lucas Toliman",
714.0:"Santa Cruz La Laguna",
715.0:"San Pablo La Laguna",
716.0:"San Marcos La Laguna",
717.0:"San Juan La Laguna",
718.0:"San Pedro La Laguna",
719.0:"Santiago Atitlán",
801.0:"Totonicapán",
802.0:"San Cristóbal Totonicapán",
803.0:"San Francisco El Alto",
804.0:"San Andrés Xecul",
805.0:"Momostenango",
806.0:"Santa María Chiquimula",
807.0:"Santa Lucía La Reforma",
808.0:"San Bartolo",
901.0:"Quetzaltenango",
902.0:"Salcajá",
903.0:"Olintepeque",
904.0:"San Carlos Sija",
905.0:"Sibilia",
906.0:"Cabricán",
907.0:"Cajolá",
908.0:"San Miguel Siguilá",
909.0:"Ostuncalco",
910.0:"San Mateo",
911.0:"Concepción Chiquirichapa",
912.0:"San Martín Sacatepéquez",
913.0:"Almolonga",
914.0:"Cantel",
915.0:"Huitán",
916.0:"Zunil",
917.0:"Colomba",
918.0:"San Francisco La Unión",
919.0:"El Palmar",
920.0:"Coatepeque",
921.0:"Génova",
922.0:"Flores Costa Cuca",
923.0:"La Esperanza",
924.0:"Palestina de Los Altos",
1001.0:"Mazatenango",
1002.0:"Cuyotenango",
1003.0:"San Francisco Zapotitlán",
1004.0:"San Bernardino",
1005.0:"San José El Ídolo",
1006.0:"Santo Domingo Suchitepéquez",
1007.0:"San Lorenzo",
1008.0:"Samayac",
1009.0:"San Pablo Jocopilas",
1010.0:"San Antonio Suchitepéquez",
1011.0:"San Miguel Panán",
1012.0:"San Gabriel",
1013.0:"Chicacao",
1014.0:"Patulul",
1015.0:"Santa Bárbara",
1016.0:"San Juan Bautista",
1017.0:"Santo Tomás La Unión",
1018.0:"Zunilito",
1019.0:"Pueblo Nuevo ",
1020.0:"Rio Bravo",
1021.0:"San José La Máquina",
1101.0:"Retalhuleu",
1102.0:"San Sebastián",
1103.0:"Santa Cruz Muluá",
1104.0:"San Martín Zapotitlán",
1105.0:"San Felipe",
1106.0:"San Andrés Villa Seca",
1107.0:"Champerico",
1108.0:"Nuevo San Carlos",
1109.0:"El Asintal",
1201.0:"San Marcos",
1202.0:"San Pedro Sacatepéquez",
1203.0:"San Antonio Sacatepéquez",
1204.0:"Comitancillo",
1205.0:"San Miguel Ixtahuacán",
1206.0:"Concepción Tutuapa",
1207.0:"Tacaná",
1208.0:"Sibinal",
1209.0:"Tajumulco",
1210.0:"Tejutla",
1211.0:"San Rafael Pie de La Cuesta",
1212.0:"Nuevo Progreso",
1213.0:"El Tumbador",
1214.0:"El Rodeo",
1215.0:"Malacatán",
1216.0:"Catarina",
1217.0:"Ayutla",
1218.0:"Ocós",
1219.0:"San Pablo",
1220.0:"El Quetzal",
1221.0:"La Reforma",
1222.0:"Pajapita",
1223.0:"Ixchiguán",
1224.0:"San José Ojetenam",
1225.0:"San Cristóbal Cucho",
1226.0:"Sipacapa",
1227.0:"Esquipulas Palo Gordo",
1228.0:"Rio Blanco",
1229.0:"San Lorenzo",
1230.0:"La Blanca",
1301.0:"Huehuetenango",
1302.0:"Chiantla",
1303.0:"Malacatancito",
1304.0:"Cuilco",
1305.0:"Nentón",
1306.0:"San Pedro Necta",
1307.0:"Jacaltenango",
1308.0:"Soloma",
1309.0:"Ixtahuacán",
1310.0:"Santa Bárbara",
1311.0:"La Libertad",
1312.0:"La Democracia",
1313.0:"San Miguel Acatán",
1314.0:"San Rafael La Independencia",
1315.0:"Todos Santos Cuchumatán",
1316.0:"San Juan Atitán",
1317.0:"Santa Eulalia",
1318.0:"San Mateo Ixtatán",
1319.0:"Colotenango",
1320.0:"San Sebastián Huehuetenango",
1321.0:"Tectitán",
1322.0:"Concepción Huista",
1323.0:"San Juan Ixcoy",
1324.0:"San Antonio Huista",
1325.0:"San Sebastián Coatán",
1326.0:"Barillas",
1327.0:"Aguacatán",
1328.0:"San Rafael Petzal",
1329.0:"San Gaspar Ixchil",
1330.0:"Santiago Chimaltenango",
1331.0:"Santa Ana Huista",
1332.0:"Unión Cantinil",
1333.0:"Petatán",
1401.0:"Santa Cruz del Quiché",
1402.0:"Chiché",
1403.0:"Chinique",
1404.0:"Zacualpa",
1405.0:"Chajul",
1406.0:"Chichicastenango",
1407.0:"Patzité",
1408.0:"San Antonio Ilotenango",
1409.0:"San Pedro Jocopilas",
1410.0:"Cunén",
1411.0:"San Juan Cotzal",
1412.0:"Joyabaj",
1413.0:"Nebaj",
1414.0:"San Andrés Sajcabajá",
1415.0:"Uspantán",
1416.0:"Sacapulas",
1417.0:"San Bartolomé Jocotenango",
1418.0:"Canillá",
1419.0:"Chicamán",
1420.0:"Ixcán",
1421.0:"Pachalum",
1501.0:"Salamá",
1502.0:"San Miguel Chicaj",
1503.0:"Rabinal",
1504.0:"Cubulco",
1505.0:"Granados",
1506.0:"El Chol",
1507.0:"San Jerónimo",
1508.0:"Purulhá",
1601.0:"Cobán",
1602.0:"Santa Cruz Verapaz",
1603.0:"San Cristóbal Verapaz",
1604.0:"Tactic",
1605.0:"Tamahú",
1606.0:"Tucurú",
1607.0:"Panzós",
1608.0:"Senahú",
1609.0:"San Pedro Carchá",
1610.0:"San Juan Chamelco",
1611.0:"Lanquín",
1612.0:"Cahabón",
1613.0:"Chisec",
1614.0:"Chahal",
1615.0:"Fray Bartolomé de Las Casas",
1616.0:"Santa Catalina la Tinta",
1617.0:"Raxruhá",
1701.0:"Flores",
1702.0:"San José",
1703.0:"San Benito",
1704.0:"San Andrés",
1705.0:"La Libertad",
1706.0:"San Francisco",
1707.0:"Santa Ana",
1708.0:"Dolores",
1709.0:"San Luis",
1710.0:"Sayaxché",
1711.0:"Melchor de Mencos",
1712.0:"Poptún",
1713.0:"Las Cruces",
1714.0:"El Chal",
1801.0:"Puerto Barrios",
1802.0:"Livingston",
1803.0:"El Estor",
1804.0:"Morales",
1805.0:"Los Amates",
1901.0:"Zacapa",
1902.0:"Estanzuela",
1903.0:"Río Hondo",
1904.0:"Gualán",
1905.0:"Teculután",
1906.0:"Usumatlán",
1907.0:"Cabañas",
1908.0:"San Diego",
1909.0:"La Unión",
1910.0:"Huité",
1911.0:"San Jorge",
2001.0:"Chiquimula",
2002.0:"San José La Arada",
2003.0:"San Juan Erminta",
2004.0:"Jocotán",
2005.0:"Camotán",
2006.0:"Olopa",
2007.0:"Esquipulas",
2008.0:"Concepción Las Minas",
2009.0:"Quetzaltepeque",
2010.0:"San Jacinto",
2011.0:"Ipala",
2101.0:"Jalapa",
2102.0:"San Pedro Pinula",
2103.0:"San Luis Jilotepeque",
2104.0:"San Manuel Chaparrón",
2105.0:"San Carlos Alzatate",
2106.0:"Monjas",
2107.0:"Mataquescuintla",
2201.0:"Jutiapa",
2202.0:"El Progreso",
2203.0:"Santa Catarina Mita",
2204.0:"Agua Blanca",
2205.0:"Asunción Mita",
2206.0:"Yupiltepeque",
2207.0:"Atescatempa",
2208.0:"Jerez",
2209.0:"El Adelanto",
2210.0:"Zapotitlán",
2211.0:"Comapa",
2212.0:"Jalpatagua",
2213.0:"Conguaco",
2214.0:"Moyuta",
2215.0:"Pasaco",
2216.0:"San José Acatempa",
2217.0:"Quesada",
})

In [26]:
replace_values_in_column("HEC_DEPTOMCPIO", {
101.0:"Guatemala",
102.0:"Santa Catarina Pinula",
103.0:"San José Pinula",
104.0:"San José del Golfo",
105.0:"Palencia",
106.0:"Chinautla",
107.0:"San Pedro Ayampuc",
108.0:"Mixco",
109.0:"San Pedro Sacatepéquez",
110.0:"San Juan Sacatepéquez",
111.0:"San Raymundo",
112.0:"Chuarrancho",
113.0:"Fraijanes",
114.0:"Amatitlán",
115.0:"Villa Nueva",
116.0:"Villa Canales",
117.0:"Petapa",
201.0:"Guastatoya",
202.0:"Morazán",
203.0:"San Agustín Acasaguastlán",
204.0:"San Cristóbal Acasaguastlán",
205.0:"El Jícaro",
206.0:"Sansare",
207.0:"Sanarate",
208.0:"San Antonio La Paz",
301.0:"Antigua Guatemala",
302.0:"Jocotenango",
303.0:"Pastores",
304.0:"Sumpango",
305.0:"Santo Domingo Xenacoj",
306.0:"Santiago Sacatepéquez",
307.0:"San Bartolomé Milpas Altas",
308.0:"San Lucas Sacatepéquez",
309.0:"Santa Lucía Milpas Altas",
310.0:"Magdalena Milpas Altas",
311.0:"Santa María de Jesús",
312.0:"Ciudad Vieja",
313.0:"San Miguel Dueñas",
314.0:"Alotenango",
315.0:"San Antonio Aguas Calientes",
316.0:"Santa Catarina Barahona",
401.0:"Chimaltenango",
402.0:"San José Poaquil",
403.0:"San Martín Jilotepeque",
404.0:"Comalapa",
405.0:"Santa Apolonia",
406.0:"Tecpán Guatemala",
407.0:"Patzún",
408.0:"Pochuta",
409.0:"Patzicía",
410.0:"Santa Cruz Balanyá",
411.0:"Acatenango",
412.0:"Yepocapa",
413.0:"San Andrés Itzapa",
414.0:"Parramos",
415.0:"Zaragoza",
416.0:"El Tejar",
501.0:"Escuintla",
502.0:"Santa Lucía Cotzumalguapa",
503.0:"La Democracia",
504.0:"Siquinalá",
505.0:"Masagua",
506.0:"Tiquisate",
507.0:"La Gomera",
508.0:"Guanagazapa",
509.0:"San José",
510.0:"Iztapa",
511.0:"Palín",
512.0:"San Vicente Pacaya",
513.0:"Nueva Concepción",
514.0:"Sipacate",
601.0:"Cuilapa",
602.0:"Barberena",
603.0:"Santa Rosa de Lima",
604.0:"Casillas",
605.0:"San Rafael Las Flores",
606.0:"Oratorio",
607.0:"San Juan Tecuaco",
608.0:"Chiquimulilla",
609.0:"Taxisco",
610.0:"Santa María Ixhuatán",
611.0:"Guazacapán",
612.0:"Santa Cruz Naranjo",
613.0:"Pueblo Nuevo Viñas",
614.0:"Nueva Santa Rosa",
701.0:"Sololá",
702.0:"San José Chacayá",
703.0:"Santa María Visitación",
704.0:"Santa Lucía Utatlán",
705.0:"Nahualá",
706.0:"Santa Catarina Ixtahuacán",
707.0:"Santa Clara La Laguna",
708.0:"Concepción",
709.0:"San Andrés Semetabaj",
710.0:"Panajachel",
711.0:"Santa Catarina Palopó",
712.0:"San Antonio Palopó",
713.0:"San Lucas Toliman",
714.0:"Santa Cruz La Laguna",
715.0:"San Pablo La Laguna",
716.0:"San Marcos La Laguna",
717.0:"San Juan La Laguna",
718.0:"San Pedro La Laguna",
719.0:"Santiago Atitlán",
801.0:"Totonicapán",
802.0:"San Cristóbal Totonicapán",
803.0:"San Francisco El Alto",
804.0:"San Andrés Xecul",
805.0:"Momostenango",
806.0:"Santa María Chiquimula",
807.0:"Santa Lucía La Reforma",
808.0:"San Bartolo",
901.0:"Quetzaltenango",
902.0:"Salcajá",
903.0:"Olintepeque",
904.0:"San Carlos Sija",
905.0:"Sibilia",
906.0:"Cabricán",
907.0:"Cajolá",
908.0:"San Miguel Siguilá",
909.0:"Ostuncalco",
910.0:"San Mateo",
911.0:"Concepción Chiquirichapa",
912.0:"San Martín Sacatepéquez",
913.0:"Almolonga",
914.0:"Cantel",
915.0:"Huitán",
916.0:"Zunil",
917.0:"Colomba",
918.0:"San Francisco La Unión",
919.0:"El Palmar",
920.0:"Coatepeque",
921.0:"Génova",
922.0:"Flores Costa Cuca",
923.0:"La Esperanza",
924.0:"Palestina de Los Altos",
1001.0:"Mazatenango",
1002.0:"Cuyotenango",
1003.0:"San Francisco Zapotitlán",
1004.0:"San Bernardino",
1005.0:"San José El Ídolo",
1006.0:"Santo Domingo Suchitepéquez",
1007.0:"San Lorenzo",
1008.0:"Samayac",
1009.0:"San Pablo Jocopilas",
1010.0:"San Antonio Suchitepéquez",
1011.0:"San Miguel Panán",
1012.0:"San Gabriel",
1013.0:"Chicacao",
1014.0:"Patulul",
1015.0:"Santa Bárbara",
1016.0:"San Juan Bautista",
1017.0:"Santo Tomás La Unión",
1018.0:"Zunilito",
1019.0:"Pueblo Nuevo ",
1020.0:"Rio Bravo",
1021.0:"San José La Máquina",
1101.0:"Retalhuleu",
1102.0:"San Sebastián",
1103.0:"Santa Cruz Muluá",
1104.0:"San Martín Zapotitlán",
1105.0:"San Felipe",
1106.0:"San Andrés Villa Seca",
1107.0:"Champerico",
1108.0:"Nuevo San Carlos",
1109.0:"El Asintal",
1201.0:"San Marcos",
1202.0:"San Pedro Sacatepéquez",
1203.0:"San Antonio Sacatepéquez",
1204.0:"Comitancillo",
1205.0:"San Miguel Ixtahuacán",
1206.0:"Concepción Tutuapa",
1207.0:"Tacaná",
1208.0:"Sibinal",
1209.0:"Tajumulco",
1210.0:"Tejutla",
1211.0:"San Rafael Pie de La Cuesta",
1212.0:"Nuevo Progreso",
1213.0:"El Tumbador",
1214.0:"El Rodeo",
1215.0:"Malacatán",
1216.0:"Catarina",
1217.0:"Ayutla",
1218.0:"Ocós",
1219.0:"San Pablo",
1220.0:"El Quetzal",
1221.0:"La Reforma",
1222.0:"Pajapita",
1223.0:"Ixchiguán",
1224.0:"San José Ojetenam",
1225.0:"San Cristóbal Cucho",
1226.0:"Sipacapa",
1227.0:"Esquipulas Palo Gordo",
1228.0:"Rio Blanco",
1229.0:"San Lorenzo",
1230.0:"La Blanca",
1301.0:"Huehuetenango",
1302.0:"Chiantla",
1303.0:"Malacatancito",
1304.0:"Cuilco",
1305.0:"Nentón",
1306.0:"San Pedro Necta",
1307.0:"Jacaltenango",
1308.0:"Soloma",
1309.0:"Ixtahuacán",
1310.0:"Santa Bárbara",
1311.0:"La Libertad",
1312.0:"La Democracia",
1313.0:"San Miguel Acatán",
1314.0:"San Rafael La Independencia",
1315.0:"Todos Santos Cuchumatán",
1316.0:"San Juan Atitán",
1317.0:"Santa Eulalia",
1318.0:"San Mateo Ixtatán",
1319.0:"Colotenango",
1320.0:"San Sebastián Huehuetenango",
1321.0:"Tectitán",
1322.0:"Concepción Huista",
1323.0:"San Juan Ixcoy",
1324.0:"San Antonio Huista",
1325.0:"San Sebastián Coatán",
1326.0:"Barillas",
1327.0:"Aguacatán",
1328.0:"San Rafael Petzal",
1329.0:"San Gaspar Ixchil",
1330.0:"Santiago Chimaltenango",
1331.0:"Santa Ana Huista",
1332.0:"Unión Cantinil",
1333.0:"Petatán",
1401.0:"Santa Cruz del Quiché",
1402.0:"Chiché",
1403.0:"Chinique",
1404.0:"Zacualpa",
1405.0:"Chajul",
1406.0:"Chichicastenango",
1407.0:"Patzité",
1408.0:"San Antonio Ilotenango",
1409.0:"San Pedro Jocopilas",
1410.0:"Cunén",
1411.0:"San Juan Cotzal",
1412.0:"Joyabaj",
1413.0:"Nebaj",
1414.0:"San Andrés Sajcabajá",
1415.0:"Uspantán",
1416.0:"Sacapulas",
1417.0:"San Bartolomé Jocotenango",
1418.0:"Canillá",
1419.0:"Chicamán",
1420.0:"Ixcán",
1421.0:"Pachalum",
1501.0:"Salamá",
1502.0:"San Miguel Chicaj",
1503.0:"Rabinal",
1504.0:"Cubulco",
1505.0:"Granados",
1506.0:"El Chol",
1507.0:"San Jerónimo",
1508.0:"Purulhá",
1601.0:"Cobán",
1602.0:"Santa Cruz Verapaz",
1603.0:"San Cristóbal Verapaz",
1604.0:"Tactic",
1605.0:"Tamahú",
1606.0:"Tucurú",
1607.0:"Panzós",
1608.0:"Senahú",
1609.0:"San Pedro Carchá",
1610.0:"San Juan Chamelco",
1611.0:"Lanquín",
1612.0:"Cahabón",
1613.0:"Chisec",
1614.0:"Chahal",
1615.0:"Fray Bartolomé de Las Casas",
1616.0:"Santa Catalina la Tinta",
1617.0:"Raxruhá",
1701.0:"Flores",
1702.0:"San José",
1703.0:"San Benito",
1704.0:"San Andrés",
1705.0:"La Libertad",
1706.0:"San Francisco",
1707.0:"Santa Ana",
1708.0:"Dolores",
1709.0:"San Luis",
1710.0:"Sayaxché",
1711.0:"Melchor de Mencos",
1712.0:"Poptún",
1713.0:"Las Cruces",
1714.0:"El Chal",
1801.0:"Puerto Barrios",
1802.0:"Livingston",
1803.0:"El Estor",
1804.0:"Morales",
1805.0:"Los Amates",
1901.0:"Zacapa",
1902.0:"Estanzuela",
1903.0:"Río Hondo",
1904.0:"Gualán",
1905.0:"Teculután",
1906.0:"Usumatlán",
1907.0:"Cabañas",
1908.0:"San Diego",
1909.0:"La Unión",
1910.0:"Huité",
1911.0:"San Jorge",
2001.0:"Chiquimula",
2002.0:"San José La Arada",
2003.0:"San Juan Erminta",
2004.0:"Jocotán",
2005.0:"Camotán",
2006.0:"Olopa",
2007.0:"Esquipulas",
2008.0:"Concepción Las Minas",
2009.0:"Quetzaltepeque",
2010.0:"San Jacinto",
2011.0:"Ipala",
2101.0:"Jalapa",
2102.0:"San Pedro Pinula",
2103.0:"San Luis Jilotepeque",
2104.0:"San Manuel Chaparrón",
2105.0:"San Carlos Alzatate",
2106.0:"Monjas",
2107.0:"Mataquescuintla",
2201.0:"Jutiapa",
2202.0:"El Progreso",
2203.0:"Santa Catarina Mita",
2204.0:"Agua Blanca",
2205.0:"Asunción Mita",
2206.0:"Yupiltepeque",
2207.0:"Atescatempa",
2208.0:"Jerez",
2209.0:"El Adelanto",
2210.0:"Zapotitlán",
2211.0:"Comapa",
2212.0:"Jalpatagua",
2213.0:"Conguaco",
2214.0:"Moyuta",
2215.0:"Pasaco",
2216.0:"San José Acatempa",
2217.0:"Quesada",
})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [27]:
replace_values_in_column("VIC_DISC", {'Si': "Con discapacidad", "No": "Sin discapacidad"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [28]:
replace_values_in_column("VIC_OTRAS_HOM",{12.0:12,99.0:"Ignorado","01":1,"02":2,"03":3,"04":4,"05":5,"06":6,"07":7,"08":8,"09":9,"Ignorado":"Ignorado","Ninguna":0,})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [29]:
replace_values_in_column("VIC_OTRAS_MUJ",{12.0:12,99.0:"Ignorado","01":1,"02":2,"03":3,"04":4,"05":5,"06":6,"07":7,"08":8,"09":9,"Ignorado":"Ignorado","Ninguna":0,})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [30]:
replace_values_in_column("VIC_OTRAS_N_OS",{12.0:12,99.0:"Ignorado","01":1,"02":2,"03":3,"04":4,"05":5,"06":6,"07":7,"08":8,"09":9,"Ignorado":"Ignorado","Ninguna":0,})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [31]:
replace_values_in_column("VIC_OTRAS_N_AS",{12.0:12,99.0:"Ignorado","01":1,"02":2,"03":3,"04":4,"05":5,"06":6,"07":7,"08":8,"09":9,"Ignorado":"Ignorado","Ninguna":0,})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [32]:
replace_values_in_column("VIC_ALFAB", {'Si': "Alfabeta", "No": "Analfabeta"})

In [33]:
replace_values_in_column("AGR_ALFAB", {'Si': "Alfabeta", "No": "Analfabeta"})

In [34]:
replace_values_in_column(
    "AGR_ESCOLARIDAD", 
    {
        10.0: "Ninguno", 
        21.0: "Primero Primaria", 
        22.0: "Segundo Primaria", 
        23.0: "Tercero Primaria", 
        24.0: "Cuarto Primaria", 
        25.0: "Quinto Primaria", 
        26.0: "Sexto Primaria",
        29.0: "Primaria Grado Ignorado",
        31.0: "Primero Basico",
        32.0: "Segundo Basico",
        33.0: "Tercero Basico",
        39.0: "Basico Grado Ignorado",
        44.0: "Cuarto Diversificado",
        45.0: "Quinto Diversificado",
        46.0: "Sexto Diversificado",
        49.0: "Diversificado Grado Ignorado",
        51.0: "Primer Ano Universitario",
        52.0: "Segundo Ano Universitario",
        53.0: "Tercer Ano Universitario",
        54.0: "Cuarto Ano Universitario",
        55.0: "Quinto Ano Universitario",
        56.0: "Sexto Ano Universitario",
        57.0: "Septimo Ano Universitario",
        59.0: "Universitario Grado Ignorado",
        99.0: "Ignorado",
        "Cuarto año universitario": "Cuarto Ano Universitario",
        "Básico grado ignorado": "Basico Grado Ignorado",
        "Cuarto año universitario":"Cuarto Ano Universitario",
        "Cuarto diversificado":"Cuarto Diversificado",
        "Cuarto primaria":"Cuarto Primaria",
        "Diversificado grado ignorado":"Diversificado Grado Ignorado",
        "Grado universitario ignorado":"Universitario Grado Ignorado",
        "Ignorado año y nivel de escolaridad": "Ignorado",
        "Ninguna": "Ninguno",
        "Primaria grado ignorado": "Primaria Grado Ignorado",
        "Primer año universitario": "Primer Ano Universitario",
        "Primero básico": "Primero Basico",
        "Primero primaria": "Primero Primaria",
        "Quinto año universitario": "Quinto Ano Universitario",
        "Quinto diversificado": "Quinto Diversificado",
        "Quinto primaria": "Quinto Primaria",
        "Segundo año universitario": "Segundo Ano Universitario",
        "Segundo básico": "Segundo Basico",
        "Segundo primaria": "Segundo Primaria",
        "Septimo año universitario": "Septimo Ano Universitario",
        "Sexto año universitario": "Sexto Ano Universitario",
        "Sexto diversificado": "Sexto Diversificado",
        "Sexto primaria": "Sexto Primaria",
        "Tercer año universitario": "Tercer Ano Universitario",
        "Tercero básico": "Tercero Basico",
        "Tercero primaria": "Tercero Primaria",
    }
    )

In [35]:
replace_values_in_column(
    "VIC_ESCOLARIDAD", 
    {
        10.0: "Ninguno", 
        21.0: "Primero Primaria", 
        22.0: "Segundo Primaria", 
        23.0: "Tercero Primaria", 
        24.0: "Cuarto Primaria", 
        25.0: "Quinto Primaria", 
        26.0: "Sexto Primaria",
        29.0: "Primaria Grado Ignorado",
        31.0: "Primero Basico",
        32.0: "Segundo Basico",
        33.0: "Tercero Basico",
        39.0: "Basico Grado Ignorado",
        44.0: "Cuarto Diversificado",
        45.0: "Quinto Diversificado",
        46.0: "Sexto Diversificado",
        49.0: "Diversificado Grado Ignorado",
        51.0: "Primer Ano Universitario",
        52.0: "Segundo Ano Universitario",
        53.0: "Tercer Ano Universitario",
        54.0: "Cuarto Ano Universitario",
        55.0: "Quinto Ano Universitario",
        56.0: "Sexto Ano Universitario",
        57.0: "Septimo Ano Universitario",
        59.0: "Universitario Grado Ignorado",
        99.0: "Ignorado",
        "Cuarto año universitario": "Cuarto Ano Universitario",
        "Básico grado ignorado": "Basico Grado Ignorado",
        "Cuarto año universitario":"Cuarto Ano Universitario",
        "Cuarto diversificado":"Cuarto Diversificado",
        "Cuarto primaria":"Cuarto Primaria",
        "Diversificado grado ignorado":"Diversificado Grado Ignorado",
        "Grado universitario ignorado":"Universitario Grado Ignorado",
        "Ignorado año y nivel de escolaridad": "Ignorado",
        "Ninguna": "Ninguno",
        "Primaria grado ignorado": "Primaria Grado Ignorado",
        "Primer año universitario": "Primer Ano Universitario",
        "Primero básico": "Primero Basico",
        "Primero primaria": "Primero Primaria",
        "Quinto año universitario": "Quinto Ano Universitario",
        "Quinto diversificado": "Quinto Diversificado",
        "Quinto primaria": "Quinto Primaria",
        "Segundo año universitario": "Segundo Ano Universitario",
        "Segundo básico": "Segundo Basico",
        "Segundo primaria": "Segundo Primaria",
        "Septimo año universitario": "Septimo Ano Universitario",
        "Sexto año universitario": "Sexto Ano Universitario",
        "Sexto diversificado": "Sexto Diversificado",
        "Sexto primaria": "Sexto Primaria",
        "Tercer año universitario": "Tercer Ano Universitario",
        "Tercero básico": "Tercero Basico",
        "Tercero primaria": "Tercero Primaria",
    }
    )

In [36]:
replace_values_in_column("AGR_GRUPET", {'Xincas': "Xinka", "Xinca": "Xinka", "Xinkas": "Xinka", "Otros": "Otro", "Mayas": "Maya", "Garifunas":"Garifuna"})

In [37]:

replace_values_in_column("AGR_TRABAJA", {'No trabaja por un salario o ingreso': "No", "Si trabaja por un salario o ingreso": "Si"})

In [38]:
replace_values_in_column("DEPTO_MCPIO", {
101.0:"Guatemala",
102.0:"Santa Catarina Pinula",
103.0:"San José Pinula",
104.0:"San José del Golfo",
105.0:"Palencia",
106.0:"Chinautla",
107.0:"San Pedro Ayampuc",
108.0:"Mixco",
109.0:"San Pedro Sacatepéquez",
110.0:"San Juan Sacatepéquez",
111.0:"San Raymundo",
112.0:"Chuarrancho",
113.0:"Fraijanes",
114.0:"Amatitlán",
115.0:"Villa Nueva",
116.0:"Villa Canales",
117.0:"Petapa",
201.0:"Guastatoya",
202.0:"Morazán",
203.0:"San Agustín Acasaguastlán",
204.0:"San Cristóbal Acasaguastlán",
205.0:"El Jícaro",
206.0:"Sansare",
207.0:"Sanarate",
208.0:"San Antonio La Paz",
301.0:"Antigua Guatemala",
302.0:"Jocotenango",
303.0:"Pastores",
304.0:"Sumpango",
305.0:"Santo Domingo Xenacoj",
306.0:"Santiago Sacatepéquez",
307.0:"San Bartolomé Milpas Altas",
308.0:"San Lucas Sacatepéquez",
309.0:"Santa Lucía Milpas Altas",
310.0:"Magdalena Milpas Altas",
311.0:"Santa María de Jesús",
312.0:"Ciudad Vieja",
313.0:"San Miguel Dueñas",
314.0:"Alotenango",
315.0:"San Antonio Aguas Calientes",
316.0:"Santa Catarina Barahona",
401.0:"Chimaltenango",
402.0:"San José Poaquil",
403.0:"San Martín Jilotepeque",
404.0:"Comalapa",
405.0:"Santa Apolonia",
406.0:"Tecpán Guatemala",
407.0:"Patzún",
408.0:"Pochuta",
409.0:"Patzicía",
410.0:"Santa Cruz Balanyá",
411.0:"Acatenango",
412.0:"Yepocapa",
413.0:"San Andrés Itzapa",
414.0:"Parramos",
415.0:"Zaragoza",
416.0:"El Tejar",
501.0:"Escuintla",
502.0:"Santa Lucía Cotzumalguapa",
503.0:"La Democracia",
504.0:"Siquinalá",
505.0:"Masagua",
506.0:"Tiquisate",
507.0:"La Gomera",
508.0:"Guanagazapa",
509.0:"San José",
510.0:"Iztapa",
511.0:"Palín",
512.0:"San Vicente Pacaya",
513.0:"Nueva Concepción",
514.0:"Sipacate",
601.0:"Cuilapa",
602.0:"Barberena",
603.0:"Santa Rosa de Lima",
604.0:"Casillas",
605.0:"San Rafael Las Flores",
606.0:"Oratorio",
607.0:"San Juan Tecuaco",
608.0:"Chiquimulilla",
609.0:"Taxisco",
610.0:"Santa María Ixhuatán",
611.0:"Guazacapán",
612.0:"Santa Cruz Naranjo",
613.0:"Pueblo Nuevo Viñas",
614.0:"Nueva Santa Rosa",
701.0:"Sololá",
702.0:"San José Chacayá",
703.0:"Santa María Visitación",
704.0:"Santa Lucía Utatlán",
705.0:"Nahualá",
706.0:"Santa Catarina Ixtahuacán",
707.0:"Santa Clara La Laguna",
708.0:"Concepción",
709.0:"San Andrés Semetabaj",
710.0:"Panajachel",
711.0:"Santa Catarina Palopó",
712.0:"San Antonio Palopó",
713.0:"San Lucas Toliman",
714.0:"Santa Cruz La Laguna",
715.0:"San Pablo La Laguna",
716.0:"San Marcos La Laguna",
717.0:"San Juan La Laguna",
718.0:"San Pedro La Laguna",
719.0:"Santiago Atitlán",
801.0:"Totonicapán",
802.0:"San Cristóbal Totonicapán",
803.0:"San Francisco El Alto",
804.0:"San Andrés Xecul",
805.0:"Momostenango",
806.0:"Santa María Chiquimula",
807.0:"Santa Lucía La Reforma",
808.0:"San Bartolo",
901.0:"Quetzaltenango",
902.0:"Salcajá",
903.0:"Olintepeque",
904.0:"San Carlos Sija",
905.0:"Sibilia",
906.0:"Cabricán",
907.0:"Cajolá",
908.0:"San Miguel Siguilá",
909.0:"Ostuncalco",
910.0:"San Mateo",
911.0:"Concepción Chiquirichapa",
912.0:"San Martín Sacatepéquez",
913.0:"Almolonga",
914.0:"Cantel",
915.0:"Huitán",
916.0:"Zunil",
917.0:"Colomba",
918.0:"San Francisco La Unión",
919.0:"El Palmar",
920.0:"Coatepeque",
921.0:"Génova",
922.0:"Flores Costa Cuca",
923.0:"La Esperanza",
924.0:"Palestina de Los Altos",
1001.0:"Mazatenango",
1002.0:"Cuyotenango",
1003.0:"San Francisco Zapotitlán",
1004.0:"San Bernardino",
1005.0:"San José El Ídolo",
1006.0:"Santo Domingo Suchitepéquez",
1007.0:"San Lorenzo",
1008.0:"Samayac",
1009.0:"San Pablo Jocopilas",
1010.0:"San Antonio Suchitepéquez",
1011.0:"San Miguel Panán",
1012.0:"San Gabriel",
1013.0:"Chicacao",
1014.0:"Patulul",
1015.0:"Santa Bárbara",
1016.0:"San Juan Bautista",
1017.0:"Santo Tomás La Unión",
1018.0:"Zunilito",
1019.0:"Pueblo Nuevo ",
1020.0:"Rio Bravo",
1021.0:"San José La Máquina",
1101.0:"Retalhuleu",
1102.0:"San Sebastián",
1103.0:"Santa Cruz Muluá",
1104.0:"San Martín Zapotitlán",
1105.0:"San Felipe",
1106.0:"San Andrés Villa Seca",
1107.0:"Champerico",
1108.0:"Nuevo San Carlos",
1109.0:"El Asintal",
1201.0:"San Marcos",
1202.0:"San Pedro Sacatepéquez",
1203.0:"San Antonio Sacatepéquez",
1204.0:"Comitancillo",
1205.0:"San Miguel Ixtahuacán",
1206.0:"Concepción Tutuapa",
1207.0:"Tacaná",
1208.0:"Sibinal",
1209.0:"Tajumulco",
1210.0:"Tejutla",
1211.0:"San Rafael Pie de La Cuesta",
1212.0:"Nuevo Progreso",
1213.0:"El Tumbador",
1214.0:"El Rodeo",
1215.0:"Malacatán",
1216.0:"Catarina",
1217.0:"Ayutla",
1218.0:"Ocós",
1219.0:"San Pablo",
1220.0:"El Quetzal",
1221.0:"La Reforma",
1222.0:"Pajapita",
1223.0:"Ixchiguán",
1224.0:"San José Ojetenam",
1225.0:"San Cristóbal Cucho",
1226.0:"Sipacapa",
1227.0:"Esquipulas Palo Gordo",
1228.0:"Rio Blanco",
1229.0:"San Lorenzo",
1230.0:"La Blanca",
1301.0:"Huehuetenango",
1302.0:"Chiantla",
1303.0:"Malacatancito",
1304.0:"Cuilco",
1305.0:"Nentón",
1306.0:"San Pedro Necta",
1307.0:"Jacaltenango",
1308.0:"Soloma",
1309.0:"Ixtahuacán",
1310.0:"Santa Bárbara",
1311.0:"La Libertad",
1312.0:"La Democracia",
1313.0:"San Miguel Acatán",
1314.0:"San Rafael La Independencia",
1315.0:"Todos Santos Cuchumatán",
1316.0:"San Juan Atitán",
1317.0:"Santa Eulalia",
1318.0:"San Mateo Ixtatán",
1319.0:"Colotenango",
1320.0:"San Sebastián Huehuetenango",
1321.0:"Tectitán",
1322.0:"Concepción Huista",
1323.0:"San Juan Ixcoy",
1324.0:"San Antonio Huista",
1325.0:"San Sebastián Coatán",
1326.0:"Barillas",
1327.0:"Aguacatán",
1328.0:"San Rafael Petzal",
1329.0:"San Gaspar Ixchil",
1330.0:"Santiago Chimaltenango",
1331.0:"Santa Ana Huista",
1332.0:"Unión Cantinil",
1333.0:"Petatán",
1401.0:"Santa Cruz del Quiché",
1402.0:"Chiché",
1403.0:"Chinique",
1404.0:"Zacualpa",
1405.0:"Chajul",
1406.0:"Chichicastenango",
1407.0:"Patzité",
1408.0:"San Antonio Ilotenango",
1409.0:"San Pedro Jocopilas",
1410.0:"Cunén",
1411.0:"San Juan Cotzal",
1412.0:"Joyabaj",
1413.0:"Nebaj",
1414.0:"San Andrés Sajcabajá",
1415.0:"Uspantán",
1416.0:"Sacapulas",
1417.0:"San Bartolomé Jocotenango",
1418.0:"Canillá",
1419.0:"Chicamán",
1420.0:"Ixcán",
1421.0:"Pachalum",
1501.0:"Salamá",
1502.0:"San Miguel Chicaj",
1503.0:"Rabinal",
1504.0:"Cubulco",
1505.0:"Granados",
1506.0:"El Chol",
1507.0:"San Jerónimo",
1508.0:"Purulhá",
1601.0:"Cobán",
1602.0:"Santa Cruz Verapaz",
1603.0:"San Cristóbal Verapaz",
1604.0:"Tactic",
1605.0:"Tamahú",
1606.0:"Tucurú",
1607.0:"Panzós",
1608.0:"Senahú",
1609.0:"San Pedro Carchá",
1610.0:"San Juan Chamelco",
1611.0:"Lanquín",
1612.0:"Cahabón",
1613.0:"Chisec",
1614.0:"Chahal",
1615.0:"Fray Bartolomé de Las Casas",
1616.0:"Santa Catalina la Tinta",
1617.0:"Raxruhá",
1701.0:"Flores",
1702.0:"San José",
1703.0:"San Benito",
1704.0:"San Andrés",
1705.0:"La Libertad",
1706.0:"San Francisco",
1707.0:"Santa Ana",
1708.0:"Dolores",
1709.0:"San Luis",
1710.0:"Sayaxché",
1711.0:"Melchor de Mencos",
1712.0:"Poptún",
1713.0:"Las Cruces",
1714.0:"El Chal",
1801.0:"Puerto Barrios",
1802.0:"Livingston",
1803.0:"El Estor",
1804.0:"Morales",
1805.0:"Los Amates",
1901.0:"Zacapa",
1902.0:"Estanzuela",
1903.0:"Río Hondo",
1904.0:"Gualán",
1905.0:"Teculután",
1906.0:"Usumatlán",
1907.0:"Cabañas",
1908.0:"San Diego",
1909.0:"La Unión",
1910.0:"Huité",
1911.0:"San Jorge",
2001.0:"Chiquimula",
2002.0:"San José La Arada",
2003.0:"San Juan Erminta",
2004.0:"Jocotán",
2005.0:"Camotán",
2006.0:"Olopa",
2007.0:"Esquipulas",
2008.0:"Concepción Las Minas",
2009.0:"Quetzaltepeque",
2010.0:"San Jacinto",
2011.0:"Ipala",
2101.0:"Jalapa",
2102.0:"San Pedro Pinula",
2103.0:"San Luis Jilotepeque",
2104.0:"San Manuel Chaparrón",
2105.0:"San Carlos Alzatate",
2106.0:"Monjas",
2107.0:"Mataquescuintla",
2201.0:"Jutiapa",
2202.0:"El Progreso",
2203.0:"Santa Catarina Mita",
2204.0:"Agua Blanca",
2205.0:"Asunción Mita",
2206.0:"Yupiltepeque",
2207.0:"Atescatempa",
2208.0:"Jerez",
2209.0:"El Adelanto",
2210.0:"Zapotitlán",
2211.0:"Comapa",
2212.0:"Jalpatagua",
2213.0:"Conguaco",
2214.0:"Moyuta",
2215.0:"Pasaco",
2216.0:"San José Acatempa",
2217.0:"Quesada",
})

In [39]:
replace_values_in_column("HEC_DEPTOMCPIO", {
101.0:"Guatemala",
102.0:"Santa Catarina Pinula",
103.0:"San José Pinula",
104.0:"San José del Golfo",
105.0:"Palencia",
106.0:"Chinautla",
107.0:"San Pedro Ayampuc",
108.0:"Mixco",
109.0:"San Pedro Sacatepéquez",
110.0:"San Juan Sacatepéquez",
111.0:"San Raymundo",
112.0:"Chuarrancho",
113.0:"Fraijanes",
114.0:"Amatitlán",
115.0:"Villa Nueva",
116.0:"Villa Canales",
117.0:"Petapa",
201.0:"Guastatoya",
202.0:"Morazán",
203.0:"San Agustín Acasaguastlán",
204.0:"San Cristóbal Acasaguastlán",
205.0:"El Jícaro",
206.0:"Sansare",
207.0:"Sanarate",
208.0:"San Antonio La Paz",
301.0:"Antigua Guatemala",
302.0:"Jocotenango",
303.0:"Pastores",
304.0:"Sumpango",
305.0:"Santo Domingo Xenacoj",
306.0:"Santiago Sacatepéquez",
307.0:"San Bartolomé Milpas Altas",
308.0:"San Lucas Sacatepéquez",
309.0:"Santa Lucía Milpas Altas",
310.0:"Magdalena Milpas Altas",
311.0:"Santa María de Jesús",
312.0:"Ciudad Vieja",
313.0:"San Miguel Dueñas",
314.0:"Alotenango",
315.0:"San Antonio Aguas Calientes",
316.0:"Santa Catarina Barahona",
401.0:"Chimaltenango",
402.0:"San José Poaquil",
403.0:"San Martín Jilotepeque",
404.0:"Comalapa",
405.0:"Santa Apolonia",
406.0:"Tecpán Guatemala",
407.0:"Patzún",
408.0:"Pochuta",
409.0:"Patzicía",
410.0:"Santa Cruz Balanyá",
411.0:"Acatenango",
412.0:"Yepocapa",
413.0:"San Andrés Itzapa",
414.0:"Parramos",
415.0:"Zaragoza",
416.0:"El Tejar",
501.0:"Escuintla",
502.0:"Santa Lucía Cotzumalguapa",
503.0:"La Democracia",
504.0:"Siquinalá",
505.0:"Masagua",
506.0:"Tiquisate",
507.0:"La Gomera",
508.0:"Guanagazapa",
509.0:"San José",
510.0:"Iztapa",
511.0:"Palín",
512.0:"San Vicente Pacaya",
513.0:"Nueva Concepción",
514.0:"Sipacate",
601.0:"Cuilapa",
602.0:"Barberena",
603.0:"Santa Rosa de Lima",
604.0:"Casillas",
605.0:"San Rafael Las Flores",
606.0:"Oratorio",
607.0:"San Juan Tecuaco",
608.0:"Chiquimulilla",
609.0:"Taxisco",
610.0:"Santa María Ixhuatán",
611.0:"Guazacapán",
612.0:"Santa Cruz Naranjo",
613.0:"Pueblo Nuevo Viñas",
614.0:"Nueva Santa Rosa",
701.0:"Sololá",
702.0:"San José Chacayá",
703.0:"Santa María Visitación",
704.0:"Santa Lucía Utatlán",
705.0:"Nahualá",
706.0:"Santa Catarina Ixtahuacán",
707.0:"Santa Clara La Laguna",
708.0:"Concepción",
709.0:"San Andrés Semetabaj",
710.0:"Panajachel",
711.0:"Santa Catarina Palopó",
712.0:"San Antonio Palopó",
713.0:"San Lucas Toliman",
714.0:"Santa Cruz La Laguna",
715.0:"San Pablo La Laguna",
716.0:"San Marcos La Laguna",
717.0:"San Juan La Laguna",
718.0:"San Pedro La Laguna",
719.0:"Santiago Atitlán",
801.0:"Totonicapán",
802.0:"San Cristóbal Totonicapán",
803.0:"San Francisco El Alto",
804.0:"San Andrés Xecul",
805.0:"Momostenango",
806.0:"Santa María Chiquimula",
807.0:"Santa Lucía La Reforma",
808.0:"San Bartolo",
901.0:"Quetzaltenango",
902.0:"Salcajá",
903.0:"Olintepeque",
904.0:"San Carlos Sija",
905.0:"Sibilia",
906.0:"Cabricán",
907.0:"Cajolá",
908.0:"San Miguel Siguilá",
909.0:"Ostuncalco",
910.0:"San Mateo",
911.0:"Concepción Chiquirichapa",
912.0:"San Martín Sacatepéquez",
913.0:"Almolonga",
914.0:"Cantel",
915.0:"Huitán",
916.0:"Zunil",
917.0:"Colomba",
918.0:"San Francisco La Unión",
919.0:"El Palmar",
920.0:"Coatepeque",
921.0:"Génova",
922.0:"Flores Costa Cuca",
923.0:"La Esperanza",
924.0:"Palestina de Los Altos",
1001.0:"Mazatenango",
1002.0:"Cuyotenango",
1003.0:"San Francisco Zapotitlán",
1004.0:"San Bernardino",
1005.0:"San José El Ídolo",
1006.0:"Santo Domingo Suchitepéquez",
1007.0:"San Lorenzo",
1008.0:"Samayac",
1009.0:"San Pablo Jocopilas",
1010.0:"San Antonio Suchitepéquez",
1011.0:"San Miguel Panán",
1012.0:"San Gabriel",
1013.0:"Chicacao",
1014.0:"Patulul",
1015.0:"Santa Bárbara",
1016.0:"San Juan Bautista",
1017.0:"Santo Tomás La Unión",
1018.0:"Zunilito",
1019.0:"Pueblo Nuevo ",
1020.0:"Rio Bravo",
1021.0:"San José La Máquina",
1101.0:"Retalhuleu",
1102.0:"San Sebastián",
1103.0:"Santa Cruz Muluá",
1104.0:"San Martín Zapotitlán",
1105.0:"San Felipe",
1106.0:"San Andrés Villa Seca",
1107.0:"Champerico",
1108.0:"Nuevo San Carlos",
1109.0:"El Asintal",
1201.0:"San Marcos",
1202.0:"San Pedro Sacatepéquez",
1203.0:"San Antonio Sacatepéquez",
1204.0:"Comitancillo",
1205.0:"San Miguel Ixtahuacán",
1206.0:"Concepción Tutuapa",
1207.0:"Tacaná",
1208.0:"Sibinal",
1209.0:"Tajumulco",
1210.0:"Tejutla",
1211.0:"San Rafael Pie de La Cuesta",
1212.0:"Nuevo Progreso",
1213.0:"El Tumbador",
1214.0:"El Rodeo",
1215.0:"Malacatán",
1216.0:"Catarina",
1217.0:"Ayutla",
1218.0:"Ocós",
1219.0:"San Pablo",
1220.0:"El Quetzal",
1221.0:"La Reforma",
1222.0:"Pajapita",
1223.0:"Ixchiguán",
1224.0:"San José Ojetenam",
1225.0:"San Cristóbal Cucho",
1226.0:"Sipacapa",
1227.0:"Esquipulas Palo Gordo",
1228.0:"Rio Blanco",
1229.0:"San Lorenzo",
1230.0:"La Blanca",
1301.0:"Huehuetenango",
1302.0:"Chiantla",
1303.0:"Malacatancito",
1304.0:"Cuilco",
1305.0:"Nentón",
1306.0:"San Pedro Necta",
1307.0:"Jacaltenango",
1308.0:"Soloma",
1309.0:"Ixtahuacán",
1310.0:"Santa Bárbara",
1311.0:"La Libertad",
1312.0:"La Democracia",
1313.0:"San Miguel Acatán",
1314.0:"San Rafael La Independencia",
1315.0:"Todos Santos Cuchumatán",
1316.0:"San Juan Atitán",
1317.0:"Santa Eulalia",
1318.0:"San Mateo Ixtatán",
1319.0:"Colotenango",
1320.0:"San Sebastián Huehuetenango",
1321.0:"Tectitán",
1322.0:"Concepción Huista",
1323.0:"San Juan Ixcoy",
1324.0:"San Antonio Huista",
1325.0:"San Sebastián Coatán",
1326.0:"Barillas",
1327.0:"Aguacatán",
1328.0:"San Rafael Petzal",
1329.0:"San Gaspar Ixchil",
1330.0:"Santiago Chimaltenango",
1331.0:"Santa Ana Huista",
1332.0:"Unión Cantinil",
1333.0:"Petatán",
1401.0:"Santa Cruz del Quiché",
1402.0:"Chiché",
1403.0:"Chinique",
1404.0:"Zacualpa",
1405.0:"Chajul",
1406.0:"Chichicastenango",
1407.0:"Patzité",
1408.0:"San Antonio Ilotenango",
1409.0:"San Pedro Jocopilas",
1410.0:"Cunén",
1411.0:"San Juan Cotzal",
1412.0:"Joyabaj",
1413.0:"Nebaj",
1414.0:"San Andrés Sajcabajá",
1415.0:"Uspantán",
1416.0:"Sacapulas",
1417.0:"San Bartolomé Jocotenango",
1418.0:"Canillá",
1419.0:"Chicamán",
1420.0:"Ixcán",
1421.0:"Pachalum",
1501.0:"Salamá",
1502.0:"San Miguel Chicaj",
1503.0:"Rabinal",
1504.0:"Cubulco",
1505.0:"Granados",
1506.0:"El Chol",
1507.0:"San Jerónimo",
1508.0:"Purulhá",
1601.0:"Cobán",
1602.0:"Santa Cruz Verapaz",
1603.0:"San Cristóbal Verapaz",
1604.0:"Tactic",
1605.0:"Tamahú",
1606.0:"Tucurú",
1607.0:"Panzós",
1608.0:"Senahú",
1609.0:"San Pedro Carchá",
1610.0:"San Juan Chamelco",
1611.0:"Lanquín",
1612.0:"Cahabón",
1613.0:"Chisec",
1614.0:"Chahal",
1615.0:"Fray Bartolomé de Las Casas",
1616.0:"Santa Catalina la Tinta",
1617.0:"Raxruhá",
1701.0:"Flores",
1702.0:"San José",
1703.0:"San Benito",
1704.0:"San Andrés",
1705.0:"La Libertad",
1706.0:"San Francisco",
1707.0:"Santa Ana",
1708.0:"Dolores",
1709.0:"San Luis",
1710.0:"Sayaxché",
1711.0:"Melchor de Mencos",
1712.0:"Poptún",
1713.0:"Las Cruces",
1714.0:"El Chal",
1801.0:"Puerto Barrios",
1802.0:"Livingston",
1803.0:"El Estor",
1804.0:"Morales",
1805.0:"Los Amates",
1901.0:"Zacapa",
1902.0:"Estanzuela",
1903.0:"Río Hondo",
1904.0:"Gualán",
1905.0:"Teculután",
1906.0:"Usumatlán",
1907.0:"Cabañas",
1908.0:"San Diego",
1909.0:"La Unión",
1910.0:"Huité",
1911.0:"San Jorge",
2001.0:"Chiquimula",
2002.0:"San José La Arada",
2003.0:"San Juan Erminta",
2004.0:"Jocotán",
2005.0:"Camotán",
2006.0:"Olopa",
2007.0:"Esquipulas",
2008.0:"Concepción Las Minas",
2009.0:"Quetzaltepeque",
2010.0:"San Jacinto",
2011.0:"Ipala",
2101.0:"Jalapa",
2102.0:"San Pedro Pinula",
2103.0:"San Luis Jilotepeque",
2104.0:"San Manuel Chaparrón",
2105.0:"San Carlos Alzatate",
2106.0:"Monjas",
2107.0:"Mataquescuintla",
2201.0:"Jutiapa",
2202.0:"El Progreso",
2203.0:"Santa Catarina Mita",
2204.0:"Agua Blanca",
2205.0:"Asunción Mita",
2206.0:"Yupiltepeque",
2207.0:"Atescatempa",
2208.0:"Jerez",
2209.0:"El Adelanto",
2210.0:"Zapotitlán",
2211.0:"Comapa",
2212.0:"Jalpatagua",
2213.0:"Conguaco",
2214.0:"Moyuta",
2215.0:"Pasaco",
2216.0:"San José Acatempa",
2217.0:"Quesada",
})

In [40]:
# datasets["2023"]["VIC_OCUP"].value_counts()

In [41]:
# datasets["2023"][datasets["2023"]["VIC_OCUP"].str.contains('ocupacion', case=False, regex=True, na=False)]["VIC_OCUP"].value_counts()


In [42]:

replace_values_in_column("AGR_OCUP", {
1111: "Miembros del poder legislativo",
1112: "Personal directivo de la administración pública",
1113: "Jefes de pequeñas poblaciones",
1114: "Dirigentes de organizaciones que presentan un interés especial",
1120: "Directores generales y gerentes generales",
1211: "Directores financieros",
1212: "Directores de recursos humanos",
1213: "Directores de políticas y planificación",
1219: "Directores de administración y servicios no clasificados bajo otros epígrafes",
1221: "Directores de ventas y comercialización",
1222: "Directores de publicidad y relaciones públicas",
1223: "Directores de investigación y desarrollo",
1311: "Directores de producción agropecuaria y silvicultura",
1312: "Directores de producción de piscicultura y pesca",
1321: "Directores de industrias manufactureras",
1322: "Directores de explotaciones de minería",
1323: "Directores de empresas de construcción",
1324: "Directores de empresas de abastecimiento, distribución y afines",
1330: "Directores de servicios de tecnología de la información y las comunicaciones",
1341: "Directores de servicios de cuidados infantiles",
1342: "Directores de servicios de salud",
1343: "Directores de servicios de cuidado de las personas de edad",
1344: "Directores de servicios de bienestar social",
1345: "Directores de servicios de educación",
1346: "Gerentes de sucursales de bancos, de servicios financieros y de seguros",
1349: "Directores y gerentes de servicios profesionales no clasificados bajo otros epígrafes",
1411: "Gerentes de hoteles",
1412: "Gerentes de restaurantes",
1420: "Gerentes de comercios al por mayor y al por menor",
1431: "Gerentes de centros deportivos, de esparcimiento y culturales",
1439: "Gerentes de servicios no clasificados bajo otros epígrafes",
2111: "Físicos y astrónomos",
2112: "Meteorólogos",
2113: "Químicos",
2114: "Geólogos y geofísicos",
2120: "Matemáticos, actuarios y estadísticos",
2131: "Biólogos, botánicos, zoólogos y afines",
2132: "Agrónomos y afines",
2133: "Profesionales de la protección medioambiental",
2141: "Ingenieros industriales y de producción",
2142: "Ingenieros civiles",
2143: "Ingenieros medioambientales",
2144: "Ingenieros mecánicos",
2145: "Ingenieros químicos",
2146: "Ingenieros de minas, metalúrgicos y afines",
2149: "Ingenieros no clasificados bajo otros epígrafes",
2151: "Ingenieros electricistas",
2152: "Ingenieros electrónicos",
2153: "Ingenieros en telecomunicaciones",
2161: "Arquitectos",
2162: "Arquitectos paisajistas",
2163: "Diseñadores de productos y de prendas",
2164: "Urbanistas e ingenieros de tránsito",
2165: "Cartógrafos y agrimensores",
2166: "Diseñadores gráficos y multimedia",
2211: "Médicos generales",
2212: "Médicos especialistas",
2221: "Profesionales de enfermería",
2222: "Profesionales de partería",
2230: "Profesionales de medicina tradicional y alternativa",
2240: "Practicantes paramédicos",
2250: "Veterinarios",
2261: "Dentistas",
2262: "Farmacéuticos",
2263: "Profesionales de la salud y la higiene laboral y ambiental",
2264: "Fisioterapeutas",
2265: "Dietistas y nutricionistas",
2266: "Audiólogos y logopedas",
2267: "Optometristas",
2269: "Profesionales de la salud no clasificados bajo otros epígrafes",
2310: "Profesores de universidades y de la enseñanza superior",
2320: "Profesores de formación profesional",
2330: "Profesores de enseñanza secundaria",
2341: "Maestros de enseñanza primaria",
2342: "Maestros preescolares",
2351: "Especialistas en métodos pedagógicos",
2352: "Educadores para necesidades especiales",
2353: "Otros profesores de idiomas",
2354: "Otros profesores de música",
2355: "Otros profesores de artes",
2356: "Instructores en tecnología de la información",
2359: "Profesionales de la enseñanza no clasificados bajo otros epígrafes",
2411: "Contables",
2412: "Asesores financieros y en inversiones",
2413: "Analistas financieros",
2421: "Analistas de gestión y organización",
2422: "Especialistas en políticas de administración",
2423: "Especialistas en políticas y servicios de personal y afines",
2424: "Especialistas en formación del personal",
2431: "Profesionales de la publicidad y la comercialización",
2432: "Profesionales de relaciones públicas",
2433: "Profesionales de ventas técnicas y médicas (excluyendo la TIC)",
2434: "Profesionales de ventas de tecnología de la información y las comunicaciones",
2511: "Analistas de sistemas",
2512: "Desarrolladores de software",
2513: "Desarrolladores web y multimedia",
2514: "Programadores de aplicaciones",
2519: "Desarrolladores y analistas de software y multimedia y analistas no clasificados bajo otros epígrafes",
2521: "Diseñadores y administradores de bases de datos",
2522: "Administradores de sistemas",
2523: "Profesionales en redes de computadores",
2529: "Especialistas en bases de datos y en redes de computadores no clasificados bajo otros epígrafes",
2611: "Abogados",
2612: "Jueces",
2619: "Profesionales en derecho no clasificados bajo otros epígrafes",
2621: "Archivistas y curadores de museos",
2622: "Bibliotecarios, documentalistas y afines",
2631: "Economistas",
2632: "Sociólogos, antropólogos y afines",
2633: "Filósofos, historiadores y especialistas en ciencias políticas",
2634: "Psicólogos",
2635: "Profesionales del trabajo social",
2636: "Profesionales religiosos",
2641: "Autores y otros escritores",
2642: "Periodistas",
2643: "Traductores, intérpretes y lingüistas",
2651: "Artistas de artes plásticas",
2652: "Músicos, cantantes y compositores",
2653: "Bailarines y coreógrafos",
2654: "Directores de cine, de teatro y afines",
2655: "Actores",
2656: "Locutores de radio, televisión y otros medios de comunicación",
2659: "Artistas creativos e interpretativos no clasificados bajo otros epígrafes",
3111: "Técnicos en ciencias físicas y químicas",
3112: "Técnicos en ingeniería civil",
3113: "Electrotécnicos",
3114: "Técnicos en electrónica",
3115: "Técnicos en ingeniería mecánica",
3116: "Técnicos en química industrial",
3117: "Técnicos en ingeniería de minas y metalurgia",
3118: "Delineantes y dibujantes técnicos",
3119: "Técnicos en ciencias físicas y en ingeniería no clasificados bajo otros epígrafes",
3121: "Supervisores en ingeniería de minas",
3122: "Supervisores de industrias manufactureras",
3123: "Supervisores de la construcción",
3131: "Operadores de instalaciones de producción de energía",
3132: "Operadores de incineradores, instalaciones de tratamiento de agua y afines",
3133: "Controladores de instalaciones de procesamiento de productos químicos",
3134: "Operadores de instalaciones de refinación de petróleo y gas natural",
3135: "Controladores de procesos de producción de metales",
3139: "Técnicos en control de procesos no clasificados bajo otros epígrafes",
3141: "Técnicos en ciencias biológicas (excluyendo la medicina)",
3142: "Técnicos agropecuarios",
3143: "Técnicos forestales",
3151: "Oficiales maquinistas en navegación",
3152: "Capitanes, oficiales de cubierta y prácticos",
3153: "Pilotos de aviación y afines",
3154: "Controladores de tráfico aéreo",
3155: "Técnicos en seguridad aeronáutica",
3211: "Técnicos en aparatos de diagnóstico y tratamiento médico",
3212: "Técnicos de laboratorios médicos",
3213: "Técnicos y asistentes farmacéuticos",
3214: "Técnicos de prótesis médicas y dentales",
3221: "Profesionales de nivel medio de enfermería",
3222: "Profesionales de nivel medio de partería",
3230: "Profesionales de nivel medio de medicina tradicional y alternativa",
3240: "Técnicos y asistentes veterinarios",
3241: "Curandero",
3251: "Dentistas auxiliares y ayudantes de odontología",
3252: "Técnicos en documentación sanitaria",
3253: "Trabajadores comunitarios de la salud",
3254: "Técnicos en optometría y ópticos",
3255: "Técnicos y asistentes fisioterapeutas",
3256: "Practicantes y asistentes médicos",
3257: "Inspectores de la salud laboral, medioambiental y afines",
3258: "Ayudantes de ambulancias",
3259: "Profesionales de la salud de nivel medio no clasificados bajo otros epígrafes",
3311: "Agentes de bolsa, cambio y otros servicios financieros",
3312: "Oficiales de préstamos y créditos",
3313: "Tenedores de libros",
3314: "Profesionales de nivel medio de servicios estadísticos, matemáticos y afines",
3315: "Tasadores",
3321: "Agentes de seguros",
3322: "Representantes comerciales",
3323: "Agentes de compras",
3324: "Agentes de compras y consignatarios",
3331: "Declarantes o gestores de aduana",
3332: "Organizadores de conferencias y eventos",
3333: "Agentes de empleo y contratistas de mano de obra",
3334: "Agentes inmobiliarios",
3339: "Agentes de servicios comerciales no clasificados bajo otros epígrafes",
3341: "Supervisores de secretaría",
3342: "Secretarios jurídicos",
3343: "Secretarios administrativos y ejecutivos",
3344: "Secretarios médicos",
3351: "Agentes de aduana e inspectores de fronteras",
3352: "Agentes de administración tributaria",
3353: "Agentes de servicios de seguridad social",
3354: "Agentes de servicios de expedición de licencias y permisos",
3355: "Inspectores de policía y detectives",
3359: "Agentes de la administración pública para la aplicación de la ley y afines no clasificados bajo otros epígrafes",
3411: "Profesionales de nivel medio del derecho y servicios legales y afines",
3412: "Trabajadores y asistentes sociales de nivel medio",
3413: "Auxiliares laicos de las religiones",
3421: "Atletas y deportistas",
3422: "Entrenadores, instructores y árbitros de actividades deportivas",
3423: "Instructores de educación física y actividades recreativas",
3431: "Fotógrafos",
3432: "Diseñadores y decoradores de interior",
3433: "Técnicos en galerías de arte, museos y bibliotecas",
3434: "Chefs",
3435: "Otros profesionales de nivel medio en actividades culturales y artistas",
3511: "Técnicos en operaciones de tecnología de la información y las comunicaciones",
3512: "Técnicos en asistencia al usuario de tecnología de la información y las comunicaciones",
3513: "Técnicos en redes y sistemas de computadores",
3514: "Técnicos de la web",
3521: "Técnicos de radiodifusión y grabación audio visual",
3522: "Técnicos de ingeniería de las telecomunicaciones",
4110: "Oficinistas generales",
4120: "Secretarios (general)",
4131: "Operadores de máquinas de procesamiento de texto y mecanógrafos",
4132: "Grabadores de datos",
4211: "Cajeros de bancos y afines",
4212: "Receptores de apuestas y afines",
4213: "Prestamistas",
4214: "Cobradores y afines",
4221: "Empleados de agencias de viajes",
4222: "Empleados de centros de llamadas",
4223: "Telefonistas",
4224: "Recepcionistas de hoteles",
4225: "Empleados de ventanillas de informaciones",
4226: "Recepcionistas (general)",
4227: "Entrevistadores de encuestas y de investigaciones de mercados",
4229: "Empleados de servicios de información al cliente no clasificados bajo otros epígrafes",
4311: "Empleados de contabilidad y cálculo de costos",
4312: "Empleados de servicios estadísticos, financieros y de seguros",
4313: "Empleados encargados de las nóminas",
4321: "Empleados de control de abastecimientos e inventario",
4322: "Empleados de servicios de apoyo a la producción",
4323: "Empleados de servicios de transporte",
4411: "Empleados de bibliotecas",
4412: "Empleados de servicios de correos",
4413: "Codificadores de datos, correctores de pruebas de imprenta y afines",
4414: "Escribientes públicos y afines",
4415: "Empleados de archivos",
4416: "Empleados del servicio de personal",
4419: "Personal de apoyo administrativo no clasificado bajo otros epígrafes",
5111: "Auxiliares de servicio de abordo",
5112: "Revisores y cobradores de los transportes públicos",
5113: "Guías de turismo",
5120: "Cocineros",
5131: "Camareros de mesas",
5132: "Camareros de barra",
5141: "Peluqueros",
5142: "Especialistas en tratamientos de belleza y afines",
5151: "Supervisores de mantenimiento y limpieza en oficinas, hoteles y otros establecimientos",
5152: "Ecónomos y mayordomos domésticos",
5153: "Conserjes",
5161: "Astrólogos, adivinadores y afines",
5162: "Acompañantes y ayudantes de cámara",
5163: "Personal de pompas fúnebres y embalsamadores",
5164: "Cuidadores de animales",
5165: "Instructores de autoescuela",
5169: "Trabajadores de servicios personales no clasificados bajo otros epígrafes",
5211: "Vendedores de quioscos y de puestos de mercado",
5212: "Vendedores ambulantes de productos comestibles",
5221: "Comerciantes de tiendas",
5222: "Supervisores de tiendas y almacenes",
5223: "Asistentes de venta de tiendas y almacenes",
5230: "Cajeros y expendedores de billetes",
5241: "Modelos de moda, arte y publicidad",
5242: "Demostradores de tiendas",
5243: "Vendedores puerta a puerta",
5244: "Vendedores por teléfono",
5245: "Expendedores de gasolineras",
5246: "Vendedores de comidas al mostrador",
5249: "Vendedores no clasificados bajo otros epígrafes",
5311: "Cuidadores de niños",
5312: "Auxiliares de maestros",
5321: "Trabajadores de los cuidados personales en instituciones",
5322: "Trabajadores de los cuidados personales a domicilio",
5329: "Trabajadores de los cuidados personales en servicios de salud no clasificados bajo otros epígrafes",
5411: "Bomberos",
5412: "Policías",
5413: "Guardianes de prisión",
5414: "Guardias de protección",
5419: "Personal de los servicios de protección no clasificados bajo otros epígrafes",
6111: "Agricultores y trabajadores calificados de cultivos extensivos",
6112: "Agricultores y trabajadores calificados de plantaciones de árboles y arbustos",
6113: "Agricultores y trabajadores calificados de huertas, invernaderos, viveros y jardines",
6114: "Agricultores y trabajadores calificados de cultivos mixtos ",
6121: "Criadores de ganado",
6122: "Avicultores y trabajadores calificados de la avicultura",
6123: "Apicultores y sericultores y trabajadores calificados de la apicultura y la sericultura",
6129: "Criadores y trabajadores pecuarios calificados de la cría de animales no clasificados bajo otros epígrafes",
6130: "Productores y trabajadores calificados de explotaciones agropecuarias mixtas cuya producción se destina al mercado",
6210: "Trabajadores forestales calificados y afines",
6221: "Trabajadores de explotaciones de acuicultura",
6222: "Pescadores de agua dulce y en aguas costeras",
6223: "Pescadores de alta mar",
6224: "Cazadores y tramperos",
6310: "Trabajadores agrícolas de subsistencia",
6320: "Trabajadores pecuarios de subsistencia",
6330: "Trabajadores agropecuarios de subsistencia",
6340: "Pescadores, cazadores, tramperos y recolectores de subsistencia",
7111: "Constructores de casas",
7112: "Albañiles",
7113: "Mamposteros, tronzadores, labrantes y grabadores de piedra",
7114: "Operarios en cemento armado, enfoscadores y afines",
7115: "Carpinteros de armar y de obra blanca",
7119: "Oficiales y operarios de la construcción (obra gruesa) y afines no clasificados bajo otros epígrafes",
7121: "Techadores",
7122: "Parqueteros y colocadores de suelos",
7123: "Revocadores",
7124: "Instaladores de material aislante y de insonorización",
7125: "Cristaleros",
7126: "Fontaneros e instaladores de tuberías",
7127: "Mecánicos-montadores de instalaciones de refrigeración y climatización",
7131: "Pintores y empapeladores",
7132: "Barnizadores y afines",
7133: "Limpiadores de fachadas y deshollinadores",
7211: "Moldeadores y macheros",
7212: "Soldadores y oxicortadores",
7213: "Chapistas y caldereros",
7214: "Montadores de estructuras metálicas",
7215: "Aparejadores y empalmadores de cables",
7221: "Herreros y forjadores",
7222: "Herramentistas y afines",
7223: "Reguladores y operadores de máquinas herramientas",
7224: "Pulidores de metales y afiladores de herramientas",
7231: "Mecánicos y reparadores de vehículos de motor",
7232: "Mecánicos y reparadores de motores de avión",
7233: "Mecánicos y reparadores de máquinas agrícolas e industriales",
7234: "Reparadores de bicicletas y afines",
7311: "Mecánicos y reparadores de instrumentos de precisión",
7312: "Fabricantes y afinadores de instrumentos musicales",
7313: "Joyeros, orfebres y plateros",
7314: "Alfareros y afines (barro, arcilla y abrasivos)",
7315: "Sopladores, modeladores, laminadores, cortadores y pulidores de vidrio",
7316: "Redactores de carteles, pintores decorativos y grabadores",
7317: "Artesanos en madera, cestería y materiales similares",
7318: "Artesanos de los tejidos, el cuero y materiales similares",
7319: "Artesanos no clasificados bajo otros epígrafes",
7321: "Cajistas, tipógrafos y afines",
7322: "Impresores",
7323: "Encuadernadores y afines",
7411: "Electricistas de obras y afines",
7412: "Mecánicos y ajustadores electricistas",
7413: "Instaladores y reparadores de líneas eléctricas",
7421: "Mecánicos y reparadores en electrónica",
7422: "Instaladores y reparadores en tecnología de la información y las comunicaciones",
7511: "Carniceros, pescaderos y afines",
7512: "Panaderos, pasteleros y confiteros",
7513: "Operarios de la elaboración de productos lácteos",
7514: "Operarios de la conservación de frutas, legumbres, verduras y afines",
7515: "Catadores y clasificadores de alimentos y bebidas",
7516: "Preparadores y elaboradores de tabaco y sus productos",
7521: "Operarios del tratamiento de la madera",
7522: "Ebanistas y afines",
7523: "Reguladores y operadores de máquinas de labrar madera",
7531: "Sastres, modistos, peleteros y sombrereros",
7532: "Patronistas y cortadores de tela y afines",
7533: "Costureros, bordadores y afines",
7534: "Tapiceros, colchoneros y afines",
7535: "Apelambradores, pellejeros y curtidores",
7536: "Zapateros y afines",
7541: "Buzos",
7542: "Dinamiteros y pegadores",
7543: "Clasificadores y probadores de productos (excluyendo alimentos y bebidas)",
7544: "Fumigadores y otros controladores de plagas y malas hierbas",
7549: "Oficiales, operarios y artesanos de artes mecánicas y de otros oficios no clasificados bajo otros epígrafes",
8111: "Mineros y operadores de instalaciones mineras",
8112: "Operadores de instalaciones de procesamiento de minerales y rocas",
8113: "Perforadores y sondistas de pozos y afines",
8114: "Operadores de máquinas para fabricar cemento y otros productos minerales",
8121: "Operadores de instalaciones de procesamiento de metales",
8122: "Operadores de máquinas pulidoras, galvanizadoras y recubridoras  de metales",
8131: "Operadores de plantas y máquinas de productos químicos",
8132: "Operadores de máquinas para fabricar productos fotográficos",
8141: "Operadores de máquinas para fabricar productos de caucho",
8142: "Operadores de máquinas para fabricar productos de material plástico",
8143: "Operadores de máquinas para fabricar productos de papel",
8151: "Operadores de máquinas de preparación de fibras, hilado y devanado",
8152: "Operadores de telares y otras máquinas tejedoras",
8153: "Operadores de máquinas de coser",
8154: "Operadores de máquinas de blanqueamiento, teñido y limpieza de tejidos",
8155: "Operadores de máquinas de tratamiento de pieles y cueros",
8156: "Operadores de máquinas para la fabricación de calzado y afines",
8157: "Operadores de máquinas lavarropas",
8159: "Operadores de máquinas para fabricar productos textiles y artículos de piel y cuero no clasificados bajo otros epígrafes",
8160: "Operadores de máquinas para elaborar alimentos y productos afines",
8171: "Operadores de instalaciones para la preparación de pasta para papel y papel",
8172: "Operadores de instalaciones de procesamiento de la madera",
8181: "Operadores de instalaciones de vidriería y cerámica",
8182: "Operadores de máquinas de vapor y calderas",
8183: "Operadores de máquinas de embalaje, embotellamiento y etiquetado",
8189: "Operadores de máquinas y de instalaciones fijas no clasificados bajo otros epígrafes",
8211: "Ensambladores de maquinaria mecánica",
8212: "Ensambladores de equipos eléctricos y electrónicos",
8219: "Ensambladores no clasificados bajo otros epígrafes",
8311: "Maquinistas de locomotoras",
8312: "Guardafrenos, guardagujas y agentes de maniobras",
8321: "Conductores de motocicletas",
8322: "Conductores de automóviles, taxis y camionetas",
8331: "Conductores de autobuses y tranvías",
8332: "Conductores de camiones pesados",
8341: "Operadores de maquinaria agrícola y forestal móvil",
8342: "Operadores de máquinas de movimiento de tierras y afines",
8343: "Operadores de grúas, aparatos elevadores y afines",
8344: "Operadores de autoelevadoras",
8350: "Marineros de cubierta y afines",
9111: "Limpiadores y asistentes domésticos",
9112: "Limpiadores y asistentes de oficinas, hoteles y otros establecimientos",
9113: "Personal doméstico",
9121: "Lavanderos y planchadores manuales",
9122: "Lavadores de vehículos",
9123: "Lavadores de ventanas",
9129: "Otro personal de limpieza",
9211: "Peones de explotaciones agrícolas",
9212: "Peones de explotaciones ganaderas",
9213: "Peones de explotaciones de cultivos mixtos y ganaderos",
9214: "Peones de jardinería y horticultura",
9215: "Peones forestales",
9216: "Peones de pesca y acuicultura",
9311: "Peones de minas y canteras",
9312: "Peones de obras públicas y mantenimiento",
9313: "Peones de la construcción de edificios",
9321: "Empacadores manuales",
9329: "Peones de la industria manufacturera no clasificados bajo otros epígrafes",
9331: "Conductores de vehículos accionados a pedal o a brazo",
9332: "Conductores de vehículos y máquinas de tracción animal",
9333: "Peones de carga",
9334: "Reponedores de estanterías",
9411: "Cocineros de comidas rápidas",
9412: "Ayudantes de cocina",
9510: "Trabajadores ambulantes de servicios y afines",
9520: "Vendedores ambulantes (excluyendo de comida)",
9611: "Recolectores de basura y material reciclable",
9612: "Clasificadores de desechos",
9613: "Barrenderos y afines",
9621: "Mensajeros, mandaderos, maleteros y repartidores",
9622: "Personas que realizan trabajos varios",
9623: "Recolectores de dinero en aparatos de venta automática y lectores de medidores",
9624: "Acarreadores de agua y recolectores de leña",
9629: "Ocupaciones elementales no clasificadas bajo otros epígrafes",
"´0110": "Oficiales de las fuerzas armadas",
"´0210": "Suboficiales de las fuerzas armadas",
"´0310": "Otros miembros de las fuerzas armadas",
9998: "Ocupaciones no especificadas en las anteriores",
9999: "Ocupación ignorada",
"Ocupaciones ignoradas": "Ocupación ignorada",
3417: "Tasadores y subastadores"
})
replace_values_in_column("VIC_OCUP", {
3417: "Tasadores y subastadores",
1111: "Miembros del poder legislativo",
1112: "Personal directivo de la administración pública",
1113: "Jefes de pequeñas poblaciones",
1114: "Dirigentes de organizaciones que presentan un interés especial",
1120: "Directores generales y gerentes generales",
1211: "Directores financieros",
1212: "Directores de recursos humanos",
1213: "Directores de políticas y planificación",
1219: "Directores de administración y servicios no clasificados bajo otros epígrafes",
1221: "Directores de ventas y comercialización",
1222: "Directores de publicidad y relaciones públicas",
1223: "Directores de investigación y desarrollo",
1311: "Directores de producción agropecuaria y silvicultura",
1312: "Directores de producción de piscicultura y pesca",
1321: "Directores de industrias manufactureras",
1322: "Directores de explotaciones de minería",
1323: "Directores de empresas de construcción",
1324: "Directores de empresas de abastecimiento, distribución y afines",
1330: "Directores de servicios de tecnología de la información y las comunicaciones",
1341: "Directores de servicios de cuidados infantiles",
1342: "Directores de servicios de salud",
1343: "Directores de servicios de cuidado de las personas de edad",
1344: "Directores de servicios de bienestar social",
1345: "Directores de servicios de educación",
1346: "Gerentes de sucursales de bancos, de servicios financieros y de seguros",
1349: "Directores y gerentes de servicios profesionales no clasificados bajo otros epígrafes",
1411: "Gerentes de hoteles",
1412: "Gerentes de restaurantes",
1420: "Gerentes de comercios al por mayor y al por menor",
1431: "Gerentes de centros deportivos, de esparcimiento y culturales",
1439: "Gerentes de servicios no clasificados bajo otros epígrafes",
2111: "Físicos y astrónomos",
2112: "Meteorólogos",
2113: "Químicos",
2114: "Geólogos y geofísicos",
2120: "Matemáticos, actuarios y estadísticos",
2131: "Biólogos, botánicos, zoólogos y afines",
2132: "Agrónomos y afines",
2133: "Profesionales de la protección medioambiental",
2141: "Ingenieros industriales y de producción",
2142: "Ingenieros civiles",
2143: "Ingenieros medioambientales",
2144: "Ingenieros mecánicos",
2145: "Ingenieros químicos",
2146: "Ingenieros de minas, metalúrgicos y afines",
2149: "Ingenieros no clasificados bajo otros epígrafes",
2151: "Ingenieros electricistas",
2152: "Ingenieros electrónicos",
2153: "Ingenieros en telecomunicaciones",
2161: "Arquitectos",
2162: "Arquitectos paisajistas",
2163: "Diseñadores de productos y de prendas",
2164: "Urbanistas e ingenieros de tránsito",
2165: "Cartógrafos y agrimensores",
2166: "Diseñadores gráficos y multimedia",
2211: "Médicos generales",
2212: "Médicos especialistas",
2221: "Profesionales de enfermería",
2222: "Profesionales de partería",
2230: "Profesionales de medicina tradicional y alternativa",
2240: "Practicantes paramédicos",
2250: "Veterinarios",
2261: "Dentistas",
2262: "Farmacéuticos",
2263: "Profesionales de la salud y la higiene laboral y ambiental",
2264: "Fisioterapeutas",
2265: "Dietistas y nutricionistas",
2266: "Audiólogos y logopedas",
2267: "Optometristas",
2269: "Profesionales de la salud no clasificados bajo otros epígrafes",
2310: "Profesores de universidades y de la enseñanza superior",
2320: "Profesores de formación profesional",
2330: "Profesores de enseñanza secundaria",
2341: "Maestros de enseñanza primaria",
2342: "Maestros preescolares",
2351: "Especialistas en métodos pedagógicos",
2352: "Educadores para necesidades especiales",
2353: "Otros profesores de idiomas",
2354: "Otros profesores de música",
2355: "Otros profesores de artes",
2356: "Instructores en tecnología de la información",
2359: "Profesionales de la enseñanza no clasificados bajo otros epígrafes",
2411: "Contables",
2412: "Asesores financieros y en inversiones",
2413: "Analistas financieros",
2421: "Analistas de gestión y organización",
2422: "Especialistas en políticas de administración",
2423: "Especialistas en políticas y servicios de personal y afines",
2424: "Especialistas en formación del personal",
2431: "Profesionales de la publicidad y la comercialización",
2432: "Profesionales de relaciones públicas",
2433: "Profesionales de ventas técnicas y médicas (excluyendo la TIC)",
2434: "Profesionales de ventas de tecnología de la información y las comunicaciones",
2511: "Analistas de sistemas",
2512: "Desarrolladores de software",
2513: "Desarrolladores web y multimedia",
2514: "Programadores de aplicaciones",
2519: "Desarrolladores y analistas de software y multimedia y analistas no clasificados bajo otros epígrafes",
2521: "Diseñadores y administradores de bases de datos",
2522: "Administradores de sistemas",
2523: "Profesionales en redes de computadores",
2529: "Especialistas en bases de datos y en redes de computadores no clasificados bajo otros epígrafes",
2611: "Abogados",
2612: "Jueces",
2619: "Profesionales en derecho no clasificados bajo otros epígrafes",
2621: "Archivistas y curadores de museos",
2622: "Bibliotecarios, documentalistas y afines",
2631: "Economistas",
2632: "Sociólogos, antropólogos y afines",
2633: "Filósofos, historiadores y especialistas en ciencias políticas",
2634: "Psicólogos",
2635: "Profesionales del trabajo social",
2636: "Profesionales religiosos",
2641: "Autores y otros escritores",
2642: "Periodistas",
2643: "Traductores, intérpretes y lingüistas",
2651: "Artistas de artes plásticas",
2652: "Músicos, cantantes y compositores",
2653: "Bailarines y coreógrafos",
2654: "Directores de cine, de teatro y afines",
2655: "Actores",
2656: "Locutores de radio, televisión y otros medios de comunicación",
2659: "Artistas creativos e interpretativos no clasificados bajo otros epígrafes",
3111: "Técnicos en ciencias físicas y químicas",
3112: "Técnicos en ingeniería civil",
3113: "Electrotécnicos",
3114: "Técnicos en electrónica",
3115: "Técnicos en ingeniería mecánica",
3116: "Técnicos en química industrial",
3117: "Técnicos en ingeniería de minas y metalurgia",
3118: "Delineantes y dibujantes técnicos",
3119: "Técnicos en ciencias físicas y en ingeniería no clasificados bajo otros epígrafes",
3121: "Supervisores en ingeniería de minas",
3122: "Supervisores de industrias manufactureras",
3123: "Supervisores de la construcción",
3131: "Operadores de instalaciones de producción de energía",
3132: "Operadores de incineradores, instalaciones de tratamiento de agua y afines",
3133: "Controladores de instalaciones de procesamiento de productos químicos",
3134: "Operadores de instalaciones de refinación de petróleo y gas natural",
3135: "Controladores de procesos de producción de metales",
3139: "Técnicos en control de procesos no clasificados bajo otros epígrafes",
3141: "Técnicos en ciencias biológicas (excluyendo la medicina)",
3142: "Técnicos agropecuarios",
3143: "Técnicos forestales",
3151: "Oficiales maquinistas en navegación",
3152: "Capitanes, oficiales de cubierta y prácticos",
3153: "Pilotos de aviación y afines",
3154: "Controladores de tráfico aéreo",
3155: "Técnicos en seguridad aeronáutica",
3211: "Técnicos en aparatos de diagnóstico y tratamiento médico",
3212: "Técnicos de laboratorios médicos",
3213: "Técnicos y asistentes farmacéuticos",
3214: "Técnicos de prótesis médicas y dentales",
3221: "Profesionales de nivel medio de enfermería",
3222: "Profesionales de nivel medio de partería",
3230: "Profesionales de nivel medio de medicina tradicional y alternativa",
3240: "Técnicos y asistentes veterinarios",
3241: "Curandero",
3251: "Dentistas auxiliares y ayudantes de odontología",
3252: "Técnicos en documentación sanitaria",
3253: "Trabajadores comunitarios de la salud",
3254: "Técnicos en optometría y ópticos",
3255: "Técnicos y asistentes fisioterapeutas",
3256: "Practicantes y asistentes médicos",
3257: "Inspectores de la salud laboral, medioambiental y afines",
3258: "Ayudantes de ambulancias",
3259: "Profesionales de la salud de nivel medio no clasificados bajo otros epígrafes",
3311: "Agentes de bolsa, cambio y otros servicios financieros",
3312: "Oficiales de préstamos y créditos",
3313: "Tenedores de libros",
3314: "Profesionales de nivel medio de servicios estadísticos, matemáticos y afines",
3315: "Tasadores",
3321: "Agentes de seguros",
3322: "Representantes comerciales",
3323: "Agentes de compras",
3324: "Agentes de compras y consignatarios",
3331: "Declarantes o gestores de aduana",
3332: "Organizadores de conferencias y eventos",
3333: "Agentes de empleo y contratistas de mano de obra",
3334: "Agentes inmobiliarios",
3339: "Agentes de servicios comerciales no clasificados bajo otros epígrafes",
3341: "Supervisores de secretaría",
3342: "Secretarios jurídicos",
3343: "Secretarios administrativos y ejecutivos",
3344: "Secretarios médicos",
3351: "Agentes de aduana e inspectores de fronteras",
3352: "Agentes de administración tributaria",
3353: "Agentes de servicios de seguridad social",
3354: "Agentes de servicios de expedición de licencias y permisos",
3355: "Inspectores de policía y detectives",
3359: "Agentes de la administración pública para la aplicación de la ley y afines no clasificados bajo otros epígrafes",
3411: "Profesionales de nivel medio del derecho y servicios legales y afines",
3412: "Trabajadores y asistentes sociales de nivel medio",
3413: "Auxiliares laicos de las religiones",
3421: "Atletas y deportistas",
3422: "Entrenadores, instructores y árbitros de actividades deportivas",
3423: "Instructores de educación física y actividades recreativas",
3431: "Fotógrafos",
3432: "Diseñadores y decoradores de interior",
3433: "Técnicos en galerías de arte, museos y bibliotecas",
3434: "Chefs",
3435: "Otros profesionales de nivel medio en actividades culturales y artistas",
3511: "Técnicos en operaciones de tecnología de la información y las comunicaciones",
3512: "Técnicos en asistencia al usuario de tecnología de la información y las comunicaciones",
3513: "Técnicos en redes y sistemas de computadores",
3514: "Técnicos de la web",
3521: "Técnicos de radiodifusión y grabación audio visual",
3522: "Técnicos de ingeniería de las telecomunicaciones",
4110: "Oficinistas generales",
4120: "Secretarios (general)",
4131: "Operadores de máquinas de procesamiento de texto y mecanógrafos",
4132: "Grabadores de datos",
4211: "Cajeros de bancos y afines",
4212: "Receptores de apuestas y afines",
4213: "Prestamistas",
4214: "Cobradores y afines",
4221: "Empleados de agencias de viajes",
4222: "Empleados de centros de llamadas",
4223: "Telefonistas",
4224: "Recepcionistas de hoteles",
4225: "Empleados de ventanillas de informaciones",
4226: "Recepcionistas (general)",
4227: "Entrevistadores de encuestas y de investigaciones de mercados",
4229: "Empleados de servicios de información al cliente no clasificados bajo otros epígrafes",
4311: "Empleados de contabilidad y cálculo de costos",
4312: "Empleados de servicios estadísticos, financieros y de seguros",
4313: "Empleados encargados de las nóminas",
4321: "Empleados de control de abastecimientos e inventario",
4322: "Empleados de servicios de apoyo a la producción",
4323: "Empleados de servicios de transporte",
4411: "Empleados de bibliotecas",
4412: "Empleados de servicios de correos",
4413: "Codificadores de datos, correctores de pruebas de imprenta y afines",
4414: "Escribientes públicos y afines",
4415: "Empleados de archivos",
4416: "Empleados del servicio de personal",
4419: "Personal de apoyo administrativo no clasificado bajo otros epígrafes",
5111: "Auxiliares de servicio de abordo",
5112: "Revisores y cobradores de los transportes públicos",
5113: "Guías de turismo",
5120: "Cocineros",
5131: "Camareros de mesas",
5132: "Camareros de barra",
5141: "Peluqueros",
5142: "Especialistas en tratamientos de belleza y afines",
5151: "Supervisores de mantenimiento y limpieza en oficinas, hoteles y otros establecimientos",
5152: "Ecónomos y mayordomos domésticos",
5153: "Conserjes",
5161: "Astrólogos, adivinadores y afines",
5162: "Acompañantes y ayudantes de cámara",
5163: "Personal de pompas fúnebres y embalsamadores",
5164: "Cuidadores de animales",
5165: "Instructores de autoescuela",
5169: "Trabajadores de servicios personales no clasificados bajo otros epígrafes",
5211: "Vendedores de quioscos y de puestos de mercado",
5212: "Vendedores ambulantes de productos comestibles",
5221: "Comerciantes de tiendas",
5222: "Supervisores de tiendas y almacenes",
5223: "Asistentes de venta de tiendas y almacenes",
5230: "Cajeros y expendedores de billetes",
5241: "Modelos de moda, arte y publicidad",
5242: "Demostradores de tiendas",
5243: "Vendedores puerta a puerta",
5244: "Vendedores por teléfono",
5245: "Expendedores de gasolineras",
5246: "Vendedores de comidas al mostrador",
5249: "Vendedores no clasificados bajo otros epígrafes",
5311: "Cuidadores de niños",
5312: "Auxiliares de maestros",
5321: "Trabajadores de los cuidados personales en instituciones",
5322: "Trabajadores de los cuidados personales a domicilio",
5329: "Trabajadores de los cuidados personales en servicios de salud no clasificados bajo otros epígrafes",
5411: "Bomberos",
5412: "Policías",
5413: "Guardianes de prisión",
5414: "Guardias de protección",
5419: "Personal de los servicios de protección no clasificados bajo otros epígrafes",
6111: "Agricultores y trabajadores calificados de cultivos extensivos",
6112: "Agricultores y trabajadores calificados de plantaciones de árboles y arbustos",
6113: "Agricultores y trabajadores calificados de huertas, invernaderos, viveros y jardines",
6114: "Agricultores y trabajadores calificados de cultivos mixtos ",
6121: "Criadores de ganado",
6122: "Avicultores y trabajadores calificados de la avicultura",
6123: "Apicultores y sericultores y trabajadores calificados de la apicultura y la sericultura",
6129: "Criadores y trabajadores pecuarios calificados de la cría de animales no clasificados bajo otros epígrafes",
6130: "Productores y trabajadores calificados de explotaciones agropecuarias mixtas cuya producción se destina al mercado",
6210: "Trabajadores forestales calificados y afines",
6221: "Trabajadores de explotaciones de acuicultura",
6222: "Pescadores de agua dulce y en aguas costeras",
6223: "Pescadores de alta mar",
6224: "Cazadores y tramperos",
6310: "Trabajadores agrícolas de subsistencia",
6320: "Trabajadores pecuarios de subsistencia",
6330: "Trabajadores agropecuarios de subsistencia",
6340: "Pescadores, cazadores, tramperos y recolectores de subsistencia",
7111: "Constructores de casas",
7112: "Albañiles",
7113: "Mamposteros, tronzadores, labrantes y grabadores de piedra",
7114: "Operarios en cemento armado, enfoscadores y afines",
7115: "Carpinteros de armar y de obra blanca",
7119: "Oficiales y operarios de la construcción (obra gruesa) y afines no clasificados bajo otros epígrafes",
7121: "Techadores",
7122: "Parqueteros y colocadores de suelos",
7123: "Revocadores",
7124: "Instaladores de material aislante y de insonorización",
7125: "Cristaleros",
7126: "Fontaneros e instaladores de tuberías",
7127: "Mecánicos-montadores de instalaciones de refrigeración y climatización",
7131: "Pintores y empapeladores",
7132: "Barnizadores y afines",
7133: "Limpiadores de fachadas y deshollinadores",
7211: "Moldeadores y macheros",
7212: "Soldadores y oxicortadores",
7213: "Chapistas y caldereros",
7214: "Montadores de estructuras metálicas",
7215: "Aparejadores y empalmadores de cables",
7221: "Herreros y forjadores",
7222: "Herramentistas y afines",
7223: "Reguladores y operadores de máquinas herramientas",
7224: "Pulidores de metales y afiladores de herramientas",
7231: "Mecánicos y reparadores de vehículos de motor",
7232: "Mecánicos y reparadores de motores de avión",
7233: "Mecánicos y reparadores de máquinas agrícolas e industriales",
7234: "Reparadores de bicicletas y afines",
7311: "Mecánicos y reparadores de instrumentos de precisión",
7312: "Fabricantes y afinadores de instrumentos musicales",
7313: "Joyeros, orfebres y plateros",
7314: "Alfareros y afines (barro, arcilla y abrasivos)",
7315: "Sopladores, modeladores, laminadores, cortadores y pulidores de vidrio",
7316: "Redactores de carteles, pintores decorativos y grabadores",
7317: "Artesanos en madera, cestería y materiales similares",
7318: "Artesanos de los tejidos, el cuero y materiales similares",
7319: "Artesanos no clasificados bajo otros epígrafes",
7321: "Cajistas, tipógrafos y afines",
7322: "Impresores",
7323: "Encuadernadores y afines",
7411: "Electricistas de obras y afines",
7412: "Mecánicos y ajustadores electricistas",
7413: "Instaladores y reparadores de líneas eléctricas",
7421: "Mecánicos y reparadores en electrónica",
7422: "Instaladores y reparadores en tecnología de la información y las comunicaciones",
7511: "Carniceros, pescaderos y afines",
7512: "Panaderos, pasteleros y confiteros",
7513: "Operarios de la elaboración de productos lácteos",
7514: "Operarios de la conservación de frutas, legumbres, verduras y afines",
7515: "Catadores y clasificadores de alimentos y bebidas",
7516: "Preparadores y elaboradores de tabaco y sus productos",
7521: "Operarios del tratamiento de la madera",
7522: "Ebanistas y afines",
7523: "Reguladores y operadores de máquinas de labrar madera",
7531: "Sastres, modistos, peleteros y sombrereros",
7532: "Patronistas y cortadores de tela y afines",
7533: "Costureros, bordadores y afines",
7534: "Tapiceros, colchoneros y afines",
7535: "Apelambradores, pellejeros y curtidores",
7536: "Zapateros y afines",
7541: "Buzos",
7542: "Dinamiteros y pegadores",
7543: "Clasificadores y probadores de productos (excluyendo alimentos y bebidas)",
7544: "Fumigadores y otros controladores de plagas y malas hierbas",
7549: "Oficiales, operarios y artesanos de artes mecánicas y de otros oficios no clasificados bajo otros epígrafes",
8111: "Mineros y operadores de instalaciones mineras",
8112: "Operadores de instalaciones de procesamiento de minerales y rocas",
8113: "Perforadores y sondistas de pozos y afines",
8114: "Operadores de máquinas para fabricar cemento y otros productos minerales",
8121: "Operadores de instalaciones de procesamiento de metales",
8122: "Operadores de máquinas pulidoras, galvanizadoras y recubridoras  de metales",
8131: "Operadores de plantas y máquinas de productos químicos",
8132: "Operadores de máquinas para fabricar productos fotográficos",
8141: "Operadores de máquinas para fabricar productos de caucho",
8142: "Operadores de máquinas para fabricar productos de material plástico",
8143: "Operadores de máquinas para fabricar productos de papel",
8151: "Operadores de máquinas de preparación de fibras, hilado y devanado",
8152: "Operadores de telares y otras máquinas tejedoras",
8153: "Operadores de máquinas de coser",
8154: "Operadores de máquinas de blanqueamiento, teñido y limpieza de tejidos",
8155: "Operadores de máquinas de tratamiento de pieles y cueros",
8156: "Operadores de máquinas para la fabricación de calzado y afines",
8157: "Operadores de máquinas lavarropas",
8159: "Operadores de máquinas para fabricar productos textiles y artículos de piel y cuero no clasificados bajo otros epígrafes",
8160: "Operadores de máquinas para elaborar alimentos y productos afines",
8171: "Operadores de instalaciones para la preparación de pasta para papel y papel",
8172: "Operadores de instalaciones de procesamiento de la madera",
8181: "Operadores de instalaciones de vidriería y cerámica",
8182: "Operadores de máquinas de vapor y calderas",
8183: "Operadores de máquinas de embalaje, embotellamiento y etiquetado",
8189: "Operadores de máquinas y de instalaciones fijas no clasificados bajo otros epígrafes",
8211: "Ensambladores de maquinaria mecánica",
8212: "Ensambladores de equipos eléctricos y electrónicos",
8219: "Ensambladores no clasificados bajo otros epígrafes",
8311: "Maquinistas de locomotoras",
8312: "Guardafrenos, guardagujas y agentes de maniobras",
8321: "Conductores de motocicletas",
8322: "Conductores de automóviles, taxis y camionetas",
8331: "Conductores de autobuses y tranvías",
8332: "Conductores de camiones pesados",
8341: "Operadores de maquinaria agrícola y forestal móvil",
8342: "Operadores de máquinas de movimiento de tierras y afines",
8343: "Operadores de grúas, aparatos elevadores y afines",
8344: "Operadores de autoelevadoras",
8350: "Marineros de cubierta y afines",
9111: "Limpiadores y asistentes domésticos",
9112: "Limpiadores y asistentes de oficinas, hoteles y otros establecimientos",
9113: "Personal doméstico",
9121: "Lavanderos y planchadores manuales",
9122: "Lavadores de vehículos",
9123: "Lavadores de ventanas",
9129: "Otro personal de limpieza",
9211: "Peones de explotaciones agrícolas",
9212: "Peones de explotaciones ganaderas",
9213: "Peones de explotaciones de cultivos mixtos y ganaderos",
9214: "Peones de jardinería y horticultura",
9215: "Peones forestales",
9216: "Peones de pesca y acuicultura",
9311: "Peones de minas y canteras",
9312: "Peones de obras públicas y mantenimiento",
9313: "Peones de la construcción de edificios",
9321: "Empacadores manuales",
9329: "Peones de la industria manufacturera no clasificados bajo otros epígrafes",
9331: "Conductores de vehículos accionados a pedal o a brazo",
9332: "Conductores de vehículos y máquinas de tracción animal",
9333: "Peones de carga",
9334: "Reponedores de estanterías",
9411: "Cocineros de comidas rápidas",
9412: "Ayudantes de cocina",
9510: "Trabajadores ambulantes de servicios y afines",
9520: "Vendedores ambulantes (excluyendo de comida)",
9611: "Recolectores de basura y material reciclable",
9612: "Clasificadores de desechos",
9613: "Barrenderos y afines",
9621: "Mensajeros, mandaderos, maleteros y repartidores",
9622: "Personas que realizan trabajos varios",
9623: "Recolectores de dinero en aparatos de venta automática y lectores de medidores",
9624: "Acarreadores de agua y recolectores de leña",
9629: "Ocupaciones elementales no clasificadas bajo otros epígrafes",
"´0110": "Oficiales de las fuerzas armadas",
"´0210": "Suboficiales de las fuerzas armadas",
"´0310": "Otros miembros de las fuerzas armadas",
9998: "Ocupaciones no especificadas en las anteriores",
9999: "Ocupación ignorada",
"Ocupaciones ignoradas": "Ocupación ignorada"
})


C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [43]:
replace_values_in_column("HEC_DIA", {"Ignorado":"99"})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [44]:
replace_values_in_column("ARTICULOVCM1", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVCM2", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVCM3", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVCM4", {"Ignorado":"999.0", "Blanco":-1.0})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [45]:
replace_values_in_column("ARTICULOVIF1", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVIF2", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVIF3", {"Ignorado":"999.0", "Blanco":-1.0})
replace_values_in_column("ARTICULOVIF4", {"Ignorado":"999.0", "Blanco":-1.0})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [46]:
replace_values_in_column("ARTICULOCODPEN1",{ "Blanco":-1.0, "Ignorado": 9999})
replace_values_in_column("ARTICULOCODPEN2",{ "Blanco":-1.0, "Ignorado": 9999})
replace_values_in_column("ARTICULOCODPEN3",{ "Blanco":-1.0, "Ignorado": 9999})
replace_values_in_column("ARTICULOCODPEN4",{ "Blanco":-1.0, "Ignorado": 9999})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [47]:
replace_values_in_column("NUM_HIJ_HOM",{ "Ninguna":0.0, "Ignorado": 9999})
replace_values_in_column("NUM_HIJ_MUJ",{ "Ninguna":0.0, "Ignorado": 9999})
replace_values_in_column("OTRAS_VICTIMAS",{ "Ninguna":0.0, "Ignorado": 9999})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [48]:
replace_values_in_column("AGR_EDAD", {"Ignorado": 9999, "98 y más": 98})
replace_values_in_column("VIC_EDAD", {"Ignorado": 9999, "98 y más": 98})
replace_values_in_column("HEC_ANO", {"Ignorado": 9999})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [49]:
replace_values_in_column("TOTAL_HIJOS",{ "Ninguna":0.0, "Ignorado": 9999})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [50]:
replace_values_in_column("VIC_OTRAS_HOM",{ "Ninguna":0.0, "Ignorado": 9999})
replace_values_in_column("VIC_OTRAS_MUJ",{ "Ninguna":0.0, "Ignorado": 9999})
replace_values_in_column("VIC_OTRAS_N_AS",{ "Ninguna":0.0, "Ignorado": 9999})
replace_values_in_column("VIC_OTRAS_N_OS",{ "Ninguna":0.0, "Ignorado": 9999})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [51]:
replace_values_in_column("LEY_APLICABLE",{
    "AMBAS":"AMBAS_LEYES_VIF/VCM",
    "Ambas leyes (VIF y VCM)":"AMBAS_LEYES_VIF/VCM",
    "Ignorado":"IGNORADO",
    "Ley VCM ó Decreto 22-2008": "LEY_VCM",
    "VCM":"LEY_VCM",
    "VIF":"LEY_VIF",
    "Ley VIF ó Decreto 97-96":"LEY_VIF",
    "VIF/CPENAL":"CODIGO_PENAL"
})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [52]:
replace_values_in_column("ORGANISMO_JURISDIRECCIONAL",{
1.0:"Juzgados de Paz",
2.0:"Juzgados de Paz Comunitarios",
3.0:"Juzgados de Paz Penal ",
4.0:"Juzgado de Paz Penal de Conocimiento a Prevención de Delitos de Narcotráfico, Defraudación y Contrabando Aduanero, Violencia Sexual, Explotación y Trata de Personas ",
5.0:"Juzgados de Paz Penal de Turno/24horas",
6.0:"Juzgados de Paz Móvil",
7.0:"Juzgado de Paz con Competencia Específica para la Protección en Materia de Violencia Intrafamiliar, y de Niñez y Adolescencia Amenazada o Violada en sus Derechos (de turno)",
8.0:"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente",
9.0:"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente de Turno/24 horas",
10.0:"Juzgado de Turno de Primera Instancia Penal de Delitos de Femicidio y otras formas de de Violencia contra la contra la Mujer y Violencia Sexual, Explotación y Trata de Personas",
11.0:"Juzgados de Primera Instancia Penal de Delitos de Femicidio y otras formas de Violencia contra la Mujer",
12.0:"Tribunales de Sentencia Penal, Narcoactividad y Delitos contra el Ambiente",
13.0:"Tribunales de Sentencia Penal de Delitos de Femicidio y otras formas de Violencia contra la Mujer",
14.0:"Juzgados de Primera Instancia Ramo Mixto",
15.0:"Juzgados de Primera Instancia de Familia",
16.0:"Juzgados de Adolescentes en Conflicto con la Ley Penal",
99.0:"Ignorado",
})

In [53]:
replace_values_in_column("AGR_OTRAS_MUJ", {"Ignorado":99.0, "Ninguna":0})
replace_values_in_column("AGR_OTROS_HOM", {"Ignorado":99.0, "Ninguna":0})
replace_values_in_column("AGRESORES_OTROS_TOTAL", {"Ignorado":99.0,"Ninguna":0})
replace_values_in_column("AGR_OTRAS_N_AS", {"Ignorado":99.0,"Ninguna":0})
replace_values_in_column("AGR_OTROS_N_OS", {"Ignorado":99.0,"Ninguna":0})

C:\Users\diego\AppData\Local\Temp\ipykernel_14980\129711223.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dataset[column] = dataset[column].replace(values)


In [54]:
replace_values_in_column("ORGANISMO_REMITE",
{
    1.0:"Juzgados de Paz",
    2.0:"Juzgados de Paz Comunitarios",
    3.0:"Juzgados de Paz Penal ",
    4.0:"Juzgado de Paz Penal de Conocimiento a Prevención de Delitos de Narcotráfico, Defraudación y Contrabando Aduanero, Violencia Sexual, Explotación y Trata de Personas ",
    5.0:"Juzgados de Paz Penal de Turno/24horas",
    6.0:"Juzgados de Paz Móvil",
    7.0:"Juzgado de Paz con Competencia Específica para la Protección en Materia de Violencia Intrafamiliar, y de Niñez y Adolescencia Amenazada o Violada en sus Derechos (de turno)",
    8.0:"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente",
    9.0:"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente de Turno/24 horas",
    10.0:"Juzgado de Turno de Primera Instancia Penal de Delitos de Femicidio y otras formas de  Violencia contra la contra la Mujer y Violencia Sexual, Explotación y Trata de Personas",
    11.0:"Juzgados de Primera Instancia Penal de Delitos de Femicidio y otras formas de Violencia en contra de la Mujer",
    12.0:"Tribunales de Sentencia Penal, Narcoactividad y Delitos contra el Ambiente",
    13.0:"Tribunales de Sentencia Penal de Delitos de Femicidio y otras formas de Violencia en contra de la Mujer",
    14.0:"Juzgados de Primera Instancia Ramo Mixto",
    15.0:"Juzgados de Primera Instancia de Familia",
    16.0:"Juzgados de Adolescentes en Conflicto con la Ley Penal",
    17.0:"Ministerio Público (como órgano estatal)",
    18.0:"Este mismo órgano",
    19.0: "Ignorado",
    99.0:"Ignorado",
})

In [55]:
replace_values_in_column("CONDUCENTE",{
    1.0:"Si",
    2.0:"No",
    9.0:"Ignorado"
})

## Volver a calcular los municipios y departamentos faltantes por los nulos

In [56]:
generate_department_column(datasets, dfDeptoMunic)

In [57]:
datasets["2009"]["DEPTO"].isnull().value_counts()

DEPTO
False    29178
True      2319
Name: count, dtype: int64

## Reemplazar el tipo de dato de una columna

In [58]:
def replace_data_types_of_column_in_datasets(column, data_type):
    for dataset in datasets.values():
        if column in dataset.columns:
            dataset[column] = dataset[column].astype(data_type)

In [59]:
replace_data_types_of_column_in_datasets("ARTICULOVCM1", float)
replace_data_types_of_column_in_datasets("ARTICULOVCM2", float)
replace_data_types_of_column_in_datasets("ARTICULOVCM3", float)
replace_data_types_of_column_in_datasets("ARTICULOVCM4", float)

In [60]:
replace_data_types_of_column_in_datasets("ARTICULOVIF1", float)
replace_data_types_of_column_in_datasets("ARTICULOVIF2", float)
replace_data_types_of_column_in_datasets("ARTICULOVIF3", float)
replace_data_types_of_column_in_datasets("ARTICULOVIF4", float)

In [61]:
replace_data_types_of_column_in_datasets("ARTICULOCODPEN1",float)
replace_data_types_of_column_in_datasets("ARTICULOCODPEN2",float)
replace_data_types_of_column_in_datasets("ARTICULOCODPEN3",float)
replace_data_types_of_column_in_datasets("ARTICULOCODPEN4",float)

In [62]:
replace_data_types_of_column_in_datasets("DIA_EMISION",float)

In [63]:
replace_data_types_of_column_in_datasets("NUM_HIJ_HOM",float)
replace_data_types_of_column_in_datasets("NUM_HIJ_MUJ",float)
replace_data_types_of_column_in_datasets("OTRAS_VICTIMAS",float)

In [64]:
replace_data_types_of_column_in_datasets("TOTAL_HIJOS",float)

In [65]:
replace_data_types_of_column_in_datasets("VIC_OTRAS_HOM", float)
replace_data_types_of_column_in_datasets("VIC_OTRAS_MUJ", float)
replace_data_types_of_column_in_datasets("VIC_OTRAS_N_AS", float)
replace_data_types_of_column_in_datasets("VIC_OTRAS_N_OS", float)

In [66]:
replace_data_types_of_column_in_datasets("AGR_OTRAS_MUJ", float)
replace_data_types_of_column_in_datasets("AGR_OTROS_HOM", float)
replace_data_types_of_column_in_datasets("AGRESORES_OTROS_TOTAL", float)
replace_data_types_of_column_in_datasets("AGR_OTRAS_N_AS", float)
replace_data_types_of_column_in_datasets("AGR_OTROS_N_OS", float)

In [67]:
replace_data_types_of_column_in_datasets("AGR_EDAD", float)
replace_data_types_of_column_in_datasets("VIC_EDAD", float)
replace_data_types_of_column_in_datasets("HEC_ANO", float)

## Unir columnas repetidas

In [68]:
def unify_columns_and_merge(regex):
    for name, dataset in datasets.items():
        matching_cols = [col for col in dataset.columns if re.search(regex, col)]
        if matching_cols:
            dataset[regex] = dataset[matching_cols].astype(str).agg(' '.join, axis=1).str.strip()
            dataset.drop(columns=matching_cols, inplace=True)

In [69]:

unify_columns_and_merge(r"ARTICULOVIF")

In [70]:
unify_columns_and_merge(r"ARTICULOCODPEN")

In [71]:
unify_columns_and_merge(r"ARTICULOTRAS")

In [72]:
unify_columns_and_merge(r"ARTICULOVCM")

## Unir datasets para un solo dataframe

In [73]:
df = pd.concat(datasets.values(), axis=0)
df.shape

(521955, 62)

## Unir la distribucion de bomberos

In [74]:
df_bombero = pd.read_csv(f"./dataset/variables_alternas_procesadas/distribucion_bomberos_departamentales.csv")

In [75]:
df_bombero = df_bombero.rename(
    {
        'count_municipio': 'ESTACIONES_DE_BOMBEROS_POR_MUNICIPIO', 
        'count_departamento': 'ESTACIONES_DE_BOMBEROS_POR_DEPARTAMENTO'
    },
    axis=1
)

In [76]:
df_bombero.columns

Index(['MUNICIPIO_NORM_MATCH', 'DEPARTAMENTO_NORM',
       'ESTACIONES_DE_BOMBEROS_POR_MUNICIPIO',
       'ESTACIONES_DE_BOMBEROS_POR_DEPARTAMENTO'],
      dtype='object')

In [77]:
df_bombero["MUNICIPIO_NORM_MATCH"] = df_bombero["MUNICIPIO_NORM_MATCH"].apply(lambda x: x.strip().lower() if isinstance(x, str) else x)

In [78]:

import unicodedata
def normalize_text(text):
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text
df["DEPTO_MCPIO"] = df["DEPTO_MCPIO"].apply(normalize_text)
df["HEC_DEPTOMCPIO"] = df["HEC_DEPTOMCPIO"].apply(normalize_text)
df["DEPTO"] = df["DEPTO"].apply(normalize_text)
df["HEC_DEPTO"] = df["HEC_DEPTO"].apply(normalize_text)

In [79]:
df = df.merge(df_bombero[['DEPARTAMENTO_NORM', 'ESTACIONES_DE_BOMBEROS_POR_DEPARTAMENTO']].drop_duplicates(), left_on="HEC_DEPTO", right_on="DEPARTAMENTO_NORM", how="left")

In [80]:
# Rellenar con 0 las que no hagan match
df['ESTACIONES_DE_BOMBEROS_POR_DEPARTAMENTO'] = df['ESTACIONES_DE_BOMBEROS_POR_DEPARTAMENTO'].fillna(0)
# df['ESTACIONES_DE_BOMBEROS_POR_MUNICIPIO'] = df['ESTACIONES_DE_BOMBEROS_POR_MUNICIPIO'].fillna(0)

In [81]:
df = df.drop([
       # 'MUNICIPIO_NORM_MATCH',
       'DEPARTAMENTO_NORM'],axis=1)

## Unir la distribución de servicios de justicia

In [82]:
df_justicia = pd.read_csv(f"./dataset/variables_alternas_procesadas/distribucion_de_servicios_de_justicia.csv")

In [83]:
len(df_justicia[['DEPARTAMENTO_NORM', 'Total']].drop_duplicates())

22

In [84]:
len(df_justicia[['DEPARTAMENTO_NORM', 'TOTAL CORDINACIONES ']].drop_duplicates())

22

In [85]:
len(df_justicia[['DEPARTAMENTO_NORM', 'Totales']].drop_duplicates())

22

In [86]:
df_justicia

,Salas (mixtas),Tribunales de Sentencia,Juzgados de Primera Instancia Penal,"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente",Juzgados de Adolescentes y Niñez,Juzgados de Ejecución,Juzgados de Paz Penal y Comunitarios,Juzgados de Violencia Contra la Mujer y Femicidio,Tribunales de sentencia Violencia Contra la Mujer y Femicidio,Juzgados y Tribunales de Mayor Riesgo,...,Coordinadora municipal,Secciones especiales,TOTAL CORDINACIONES,Comisarias (oficina de atención a la víctima),Estaciones,Subestaciones,División de investigación criminal (DEIC),Totales,DEPARTAMENTO_NORM,MUNICIPIO_NORM_MATCH
0,0,2,0,1,1,0,0,1,1,0,...,0.0,0.0,1,1,4,15,1,21,chimaltenango,acatenango
1,0,3,0,2,1,0,0,1,1,0,...,0.0,0.0,1,1,6,22,1,30,jutiapa,agua blanca
2,0,2,0,1,1,0,1,1,1,0,...,0.0,0.0,6,1,7,35,1,44,huehuetenango,aguacatan
3,0,6,1,8,3,2,1,1,1,2,...,0.0,0.0,5,1,7,28,1,37,quetzaltenango,almolonga
4,1,1,1,1,1,0,1,0,0,0,...,0.0,0.0,1,1,3,17,1,22,sacatepequez,alotenango
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,1,2,0,3,1,0,0,1,1,0,...,0.0,0.0,3,1,8,23,1,33,quiche,zacualpa
329,0,1,0,1,0,0,0,0,0,0,...,0.0,0.0,1,1,3,11,1,16,el progreso,zapotitlan
330,0,2,0,1,1,0,0,1,1,0,...,0.0,0.0,1,1,4,15,1,21,chimaltenango,zaragoza
331,0,6,1,8,3,2,1,1,1,2,...,0.0,0.0,5,1,7,28,1,37,quetzaltenango,zunil


In [87]:
df_justicia = df_justicia.rename(
    {
        'Total':'CANTIDAD_ORGANISMO_JUDICIAL_POR_DEPARTAMENTO',
        'TOTAL CORDINACIONES ':'CANTIDAD_IDDP_POR_DEPARTAMENTO',
        'Totales':'CANTIDAD_PNC_POR_DEPARTAMENTO'
    }, axis=1
)

In [88]:
df = df.merge(df_justicia[[
    'Salas (mixtas)', 'Tribunales de Sentencia',
       'Juzgados de Primera Instancia Penal',
       'Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente',
       'Juzgados de Adolescentes y Niñez', 'Juzgados de Ejecución',
       'Juzgados de Paz Penal y Comunitarios',
       'Juzgados de Violencia Contra la Mujer y Femicidio',
       'Tribunales de sentencia Violencia Contra la Mujer y Femicidio',
       'Juzgados y Tribunales de Mayor Riesgo',
       'CANTIDAD_ORGANISMO_JUDICIAL_POR_DEPARTAMENTO', 'Fiscalías Regionales',
       'Fiscalías Distritales', 'Fiscalías Municipales', 'Agencias Fiscales ',
       'Coordinadora departamental', 'Coordinadora municipal',
       'Secciones especiales', 'CANTIDAD_IDDP_POR_DEPARTAMENTO',
       'Comisarias (oficina de atención a la víctima) ', 'Estaciones ',
       'Subestaciones', 'División de investigación criminal (DEIC)',
       'CANTIDAD_PNC_POR_DEPARTAMENTO', 'DEPARTAMENTO_NORM'
    ]].groupby(by=[
        'DEPARTAMENTO_NORM','CANTIDAD_PNC_POR_DEPARTAMENTO', 'CANTIDAD_ORGANISMO_JUDICIAL_POR_DEPARTAMENTO','CANTIDAD_IDDP_POR_DEPARTAMENTO'
    ]).sum().reset_index()\
    .rename(
        {
            'Salas (mixtas)':'CANTIDAD_SALAS_MIXTAS', 
            'Tribunales de Sentencia':'CANTIDAD_TRIBUNALES_DE_SENTENCIA',
            'Juzgados de Primera Instancia Penal':'CANTIDAD_JUZGADOS_PRIMERA_INSTANCIA_PENAL',
            'Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente':'CANTIDAD_JUZGADOS_PRIMERA_INSTANCIA_PENAL_NARCOACTIVIDAD_DELITOS_AMBIENTE',
            'Juzgados de Adolescentes y Niñez':'CANTIDAD_JUZGADOS_ADOLESCENTES_Y_NINEZ', 
            'Juzgados de Ejecución':'CANTIDAD_JUZGADOS_EJECUCION',
            'Juzgados de Paz Penal y Comunitarios':'CANTIDAD_JUZGADOS_PAZ_PENAL_COMUNITARIOS',
            'Juzgados de Violencia Contra la Mujer y Femicidio':'CANTIDAD_JUZGADOS_VIOLENCIA_CONTRA_MUJ_FEMINICIDIO',
            'Tribunales de sentencia Violencia Contra la Mujer y Femicidio':'CANTIDAD_TRIBUNALES_SENTENCIA_VIOLENCIA_CONTRA_MUJ_FEMINICIDIO',
            'Juzgados y Tribunales de Mayor Riesgo':'CANTIDAD_JUZGADOS_TRIBUNALES_MAYOR_RIESGO',
            'Fiscalías Regionales':'CANTIDAD_FISCALIAS_REGIONALES',
            'Fiscalías Distritales':'CANTIDAD_FISCALIAS_DISTRITALES', 
            'Fiscalías Municipales':'CANTIDAD_FISCALIAS_MUNICIPALES', 
            'Agencias Fiscales ':'CANTIDAD_AGENCIAS_FISCALES',
            'Coordinadora departamental':'CANTIDAD_COORDINADORA_DEPARTAMENTAL', 
            'Coordinadora municipal':'CANTIDAD_COORDINADORA_MUNICIPAL', 
            'Secciones especiales':'CANTIDAD_SECCIONES_ESPECIALES',
            'Comisarias (oficina de atención a la víctima) ':'CANTIDAD_COMISARIAS', 
            'Estaciones ':'CANTIDAD_ESTACIONES',
            'Subestaciones':'CANTIDAD_SUBESTACIONES', 
            'División de investigación criminal (DEIC)':'CANTIDAD_DIVISION_INVESTIGACION_CRIMINAL',
        }, axis=1
    ).drop_duplicates(), left_on="HEC_DEPTO", right_on="DEPARTAMENTO_NORM", how="left")

In [89]:
# Rellenar con 0 las que no hagan match
for col in [
    'CANTIDAD_ORGANISMO_JUDICIAL_POR_DEPARTAMENTO',
    'CANTIDAD_IDDP_POR_DEPARTAMENTO',
    'CANTIDAD_PNC_POR_DEPARTAMENTO',
    'CANTIDAD_SALAS_MIXTAS', 
    'CANTIDAD_TRIBUNALES_DE_SENTENCIA',
    'CANTIDAD_JUZGADOS_PRIMERA_INSTANCIA_PENAL',
    'CANTIDAD_JUZGADOS_PRIMERA_INSTANCIA_PENAL_NARCOACTIVIDAD_DELITOS_AMBIENTE',
    'CANTIDAD_JUZGADOS_ADOLESCENTES_Y_NINEZ', 
    'CANTIDAD_JUZGADOS_EJECUCION',
    'CANTIDAD_JUZGADOS_PAZ_PENAL_COMUNITARIOS',
    'CANTIDAD_JUZGADOS_VIOLENCIA_CONTRA_MUJ_FEMINICIDIO',
    'CANTIDAD_TRIBUNALES_SENTENCIA_VIOLENCIA_CONTRA_MUJ_FEMINICIDIO',
    'CANTIDAD_JUZGADOS_TRIBUNALES_MAYOR_RIESGO',
    'CANTIDAD_FISCALIAS_REGIONALES',
    'CANTIDAD_FISCALIAS_DISTRITALES', 
    'CANTIDAD_FISCALIAS_MUNICIPALES', 
    'CANTIDAD_AGENCIAS_FISCALES',
    'CANTIDAD_COORDINADORA_DEPARTAMENTAL', 
    'CANTIDAD_COORDINADORA_MUNICIPAL', 
    'CANTIDAD_SECCIONES_ESPECIALES',
    'CANTIDAD_COMISARIAS', 
    'CANTIDAD_ESTACIONES',
    'CANTIDAD_SUBESTACIONES', 
    'CANTIDAD_DIVISION_INVESTIGACION_CRIMINAL',
]:
    df[col] = df[col].fillna(0)
df = df.drop([
    # 'MUNICIPIO_NORM_MATCH',
       'DEPARTAMENTO_NORM'],axis=1)

## Unir la distribución de iggs

In [90]:
df_iggs = pd.read_csv(f"./dataset/variables_alternas_procesadas/distribucion_iggs.csv")

In [91]:
df_iggs.columns

Index(['DEPARTAMENTO_NORM', 'MUNICIPIO_NORM_MATCH', 'cementerio',
       'centro comunitario', 'centro de atención permanente',
       'centro de salud', 'direccion', 'escuela', 'hospital', 'otro',
       'puesto de salud'],
      dtype='object')

In [92]:
df = df.merge(df_iggs[
    ['DEPARTAMENTO_NORM','cementerio',
       'centro comunitario', 'centro de atención permanente',
       'centro de salud', 'direccion', 'escuela', 'hospital', 'otro',
       'puesto de salud']
    ].rename({
         'cementerio':'CANTIDAD_CEMENTERIO',
         'centro comunitario':'CANTIDAD_CENTRO_COMUNITARIO', 
         'centro de atención permanente':'CANTIDAD_CENTRO_ATENCION_PERMANENTE',
         'centro de salud':'CANTIDAD_CENTRO_SALUD', 
         'direccion':'CANTIDAD_DIRECCION', 
         'escuela':'CANTIDAD_ESCUELA', 
         'hospital':'CANTIDAD_HOSPITAL', 
         'otro':'CANTIDAD_OTRO',
         'puesto de salud':'CANTIDAD_PUESTO_SALUD',
      }, axis=1
   )\
   .groupby('DEPARTAMENTO_NORM').sum().reset_index()
              , left_on="HEC_DEPTO", right_on="DEPARTAMENTO_NORM", how="left")

In [96]:
df[['HEC_DEPTO', 'CANTIDAD_CEMENTERIO']].drop_duplicates()

,HEC_DEPTO,CANTIDAD_CEMENTERIO
0,guatemala,1.0
17,nan,NaN
22,ignorado,NaN
170,quetzaltenango,0.0
218,escuintla,0.0
230,huehuetenango,0.0
243,jalapa,0.0
323,el progreso,0.0
376,sacatepequez,0.0
424,chimaltenango,0.0


In [97]:
# Rellenar con 0 las que no hagan match
for col in ['CANTIDAD_CEMENTERIO',
'CANTIDAD_CENTRO_COMUNITARIO', 
'CANTIDAD_CENTRO_ATENCION_PERMANENTE',
'CANTIDAD_CENTRO_SALUD', 
'CANTIDAD_DIRECCION', 
'CANTIDAD_ESCUELA', 
'CANTIDAD_HOSPITAL', 
'CANTIDAD_OTRO',
'CANTIDAD_PUESTO_SALUD',]:
    df[col] = df[col].fillna(0)
df = df.drop([
    # 'MUNICIPIO_NORM_MATCH',
       'DEPARTAMENTO_NORM'],axis=1)

## Guardar el dataframe

In [98]:
df.to_csv(f"./dataset/violenciaIntrafamiliarGuatemala2008_2023.csv", index=False)

In [99]:
df

,AGR_ALFAB,AGR_DEDICA,AGR_EDAD,AGR_ESCOLARIDAD,AGR_EST_CIV,AGR_GRUPET,AGR_NACIONAL,AGR_OCUP,AGR_SEXO,AGR_TRABAJA,...,CANTIDAD_DIVISION_INVESTIGACION_CRIMINAL,CANTIDAD_CEMENTERIO,CANTIDAD_CENTRO_COMUNITARIO,CANTIDAD_CENTRO_ATENCION_PERMANENTE,CANTIDAD_CENTRO_SALUD,CANTIDAD_DIRECCION,CANTIDAD_ESCUELA,CANTIDAD_HOSPITAL,CANTIDAD_OTRO,CANTIDAD_PUESTO_SALUD
0,Alfabeta,NaN,25.0,Sexto Primaria,Unidos(as),Ladinos(as),Guatemalteca,Vendedores ambulantes de productos comestibles,Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
1,Alfabeta,NaN,9999.0,Basico Grado Ignorado,Casados(as),No indica,Guatemalteca,Persona de partería y enfermería,Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
2,Alfabeta,NaN,41.0,Primaria Grado Ignorado,Solteros(as),Ladinos(as),Guatemalteca,"Panaderos, pasteleros y confiteros",Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
3,Alfabeta,Desempleado,35.0,Primaria Grado Ignorado,Casados(as),Ladinos(as),Guatemalteca,NaN,Hombres,No,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
4,Alfabeta,NaN,27.0,Primaria Grado Ignorado,Casados(as),Ignorado,Guatemalteca,Vendedores ambulantes de productos no comestibles,Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521950,Ignorado,Ignorado,9999.0,Ignorado,Ignorado,Ignorado,Ignorado,Ocupación ignorada,Hombres,Ignorado,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
521951,Ignorado,Ignorado,9999.0,Ignorado,Ignorado,No indica,Extranjera,Ocupación ignorada,Mujeres,Ignorado,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
521952,Alfabeta,NaN,9999.0,Ignorado,Ignorado,Ladinos(as),Guatemalteca,"Conductores de automóviles, taxis y camionetas",Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
521953,Ignorado,NaN,9999.0,Ignorado,Ignorado,No indica,Guatemalteca,Ocupaciones no especificadas en las anteriores,Hombres,Si,...,176.0,1.0,0.0,8.0,17.0,1.0,1.0,1.0,4.0,20.0
